prepare data for pairwise plots

In [1]:
!mkdir pairwise_Fst

In [291]:
cd pairwise_Fst

/media/chrishah/STORAGE/RAD/popgen/sliding_window_plots/DIPLOTAXODON_FOR_PAPER/pairwise_Fst


In [3]:
!echo "Di_1-Di_2\nDi_1-Di_4\nDi_1-Di_5\nDi_2-Di_4\nDi_2-Di_5\nDi_4-Di_5" > comparisons.txt

In [4]:
%%bash

#specify the directory where populations had been run
stacks_dir=/media/chrishah/STORAGE/RAD/stacks/ALL/mapping/excl_PCR_dupl/BWA-8MM/M_zebra/3-populations/pairwise_single_SNP/Diplotaxodon/excl_singletons

#create symbolic links to populations fst results
for b in $(ls -1 $stacks_dir | grep "Di")
do
    ln -s $stacks_dir/$b/r_0.8-p_2-w50kb-1M_bs/batch_1.fst_$b.tsv.gz .
done


In [5]:
%%bash

#create symbolic link to matrix file from Fst summary
ln -s /media/chrishah/STORAGE/RAD/popgen/Fst-outlier/Diplotaxodon_FOR_PAPER/summarize/matrix.csv .

head matrix.csv

tag_ID,Stacks,Bayescan,Bayenv
34655,0,0,1
16078,1,0,1
19543,1,1,1
34794,0,0,1
16077,1,0,0
39952,1,0,0
22839,0,1,0
3658,1,0,0
1868,0,0,1


The next identifies windows of interest and the respective level of support for Fst candidates on all candidate scaffolds.

In [296]:
import numpy as np
import gzip


pairs = []
support = []
candidates = {}
min_support = 3
targets = {}

print targets

#record all tag ids that have been supported as candidates by at least one approach
FH = open('matrix.csv','r')
FH.readline()
for t in FH:
    columns = t.strip().split(",")
    candidates[columns[0]] = np.sum([ int(x) for x in columns[1:] ])
#print candidates
FH.close()

#record the ids of the pairwise comparisons that should be processed
FH = open('comparisons.txt','r')
for l in FH:
    pairs.append(l.strip())
FH.close()


#for each pairwise comparison produce a new text file that has an additional column with the level of support
#to be a candidate locus
for p in pairs:
    out=[]
    ids=[]
    print "processint %s" %p
    FH = gzip.open("batch_1.fst_"+p+".tsv.gz",'rb')
    out.append(FH.readline().strip().replace("# ", "")+"\tcandidate_support")
    for l in FH:
        if not l.split("\t")[1] in ids:
            ids.append(l.split("\t")[1])
            if l.split("\t")[1] in candidates: 
                out.append(l.strip()+"\t"+str(candidates[l.split("\t")[1]]))
                if candidates[l.split("\t")[1]] >= min_support:
                    if not l.split("\t")[4] in targets:
                        targets[l.split("\t")[4]] = []
                
#                targets[l.split("\t")[4]].append(l.split("\t")[5])
            else:
                out.append(l.strip()+"\t0")
    FH.close()   
    print "Done\n"
    OUT = open(p+".tsv", 'w')
    for l in out:
        OUT.write(l+"\n")
        
    OUT.close()
    for k in targets.keys():
        targets[k] = sorted(list(set(targets[k])))
#        out.append(l.strip()+"candidate_support")


#record the genomic locations of the candidates
for p in pairs:
    FH = gzip.open("batch_1.fst_"+p+".tsv.gz",'rb')
    for l in FH:
        if l.split("\t")[4] in targets:
            targets[l.split("\t")[4]].append(int(l.split("\t")[5]))
    FH.close()

#sort the relevant scaffolds by length
targets_by_length = {}
print targets.keys()
for t in sorted(targets):
    if not len(set(targets[t])) > 1:
        print "excluding %s" %t
        continue
#    print t,len(targets[t])
#    print sorted(targets[t])[0],sorted(targets[t])[-1]
    length=sorted(targets[t])[-1] - sorted(targets[t])[0]
    targets[t]=[str(length), str(sorted(targets[t])[0]), str(sorted(targets[t])[-1])]
    if not length in targets_by_length:
        targets_by_length[length] = []
    targets_by_length[length].append(t)
#    print targets[t]

longest=int(sorted(targets_by_length)[-1])
count=0


#record total lengths and some other metrics for the relevant scaffolds
cummulative = 0
for t in reversed(sorted(targets_by_length.keys())):
#    print t
    for scf in targets_by_length[t]:
#        print scf
        
        targets[scf].append(str(cummulative))
        factor="%.2f" %(float(targets[scf][0])/longest)
        targets[scf].append(factor)
        print "%s\t%s" %(scf, "\t".join(targets[scf]))#, float(targets[scf][0])/longest)
        count+=1
        cummulative+=int(targets[scf][0])

print "Total number of scaffolds to print: %s" %count

{}
processint Di_1-Di_2
Done

processint Di_1-Di_4
Done

processint Di_1-Di_5
Done

processint Di_2-Di_4
Done

processint Di_2-Di_5
Done

processint Di_4-Di_5
Done

['scaffold_242', 'scaffold_114', 'scaffold_174', 'scaffold_81', 'scaffold_45', 'scaffold_29', 'scaffold_219', 'scaffold_197']
excluding scaffold_242
scaffold_29	6356592	10019	6366611	0	1.00
scaffold_45	4371158	44680	4415838	6356592	0.69
scaffold_81	2986825	43414	3030239	10727750	0.47
scaffold_114	2313617	39730	2353347	13714575	0.36
scaffold_174	1280691	12998	1293689	16028192	0.20
scaffold_197	922914	29497	952411	17308883	0.15
scaffold_219	788691	805	789496	18231797	0.12
Total number of scaffolds to print: 7


In [297]:
regions = {}

#print candidates


for p in pairs:
    out=[]
    ids=[]
    print "processint %s" %p
    FH = open(p+".tsv", 'r')
    FH.readline()
    for l in FH:
        if l.split("\t")[1] in candidates: #if the scaffold is one of the candidates
            #extract the number of approaches supporting a given SNP as candidate locus
#            print "%s is a candidate tag" %l.split("\t")[1]
            support = l.strip().split("\t")[-1]
            if support > 0:
#                print "\n%s\t%s\t%s\t%s" %(l.split("\t")[1], l.split("\t")[4], l.split("\t")[5], support)
                if not l.split("\t")[4] in regions:
                    regions[l.split("\t")[4]] = {}
#                print regions
                for s in range(1,int(support)+1):
                    if not s in regions[l.split("\t")[4]]:
                        regions[l.split("\t")[4]][int(s)] = []
#                print regions
                for s in range(1,int(support)+1):
#                    print s
                    regions[l.split("\t")[4]][s].append(int(l.split("\t")[5]))
#                print regions
                
        
    FH.close()   
    print "Done\n"

dist=50000
windows = {}
for r in sorted(targets):#regions):
    print r
    if not r in windows:
        windows[r]={}
    for s in regions[r]:
        if not s in windows[r]:
            windows[r][s]=[]
#        print "\tsupport: %s" %s
        regions[r][s]=sorted(list(set(regions[r][s])))

        number = len(regions[r][s])
#        print "\t\t %i candidate loci - %s" %(number, str(regions[r][s]))
        lower = regions[r][s][0]-dist
        if lower < int(targets[r][1]):
            lower = int(targets[r][1])
        upper = regions[r][s][0]+dist
        if upper > int(targets[r][2]):
            upper = int(targets[r][2])
        for i in range(len(regions[r][s])-1):
#            print "\t\t%i\t%i" %(i,regions[r][s][i])
            if regions[r][s][i] + dist >= regions[r][s][i+1]-dist:
#                print "\t\t\t%i is in range - making %i the upper boundery" %(regions[r][s][i+1], regions[r][s][i+1]+dist)
                upper = regions[r][s][i+1] + dist
            else:
#                print "\t\t\t%i is NOT in range" %regions[r][s][i+1]
                windows[r][s].append([lower, upper])
                lower = regions[r][s][i+1]-dist
                if lower < int(targets[r][1]):
                    lower = int(targets[r][1])
                upper = regions[r][s][i+1]+dist
                if upper > int(targets[r][2]):
                    upper = int(targets[r][2])
        windows[r][s].append([lower, upper])
#        print "\t\t %i window(s): %s" %(len(windows[r][s]),windows[r][s])
                

                
for t in reversed(sorted(targets_by_length.keys())):
#    print t
    for scf in targets_by_length[t]:
        print "\n### %s ###\n" %scf
        for i in range(1,len(windows[scf])+1):
            print "supported by %i approach(es):" %i
            print "\t%i candidate loci - %s" %(len(regions[scf][i]), regions[scf][i])
            print "\t%i window(s) - %s" %(len(windows[scf][i]), windows[scf][i])

processint Di_1-Di_2
Done

processint Di_1-Di_4
Done

processint Di_1-Di_5
Done

processint Di_2-Di_4
Done

processint Di_2-Di_5
Done

processint Di_4-Di_5
Done

scaffold_114
scaffold_174
scaffold_197
scaffold_219
scaffold_242
scaffold_29
scaffold_45
scaffold_81

### scaffold_29 ###

supported by 1 approach(es):
	6 candidate loci - [1556890, 1759515, 1854775, 4133139, 4133150, 4133176]
	3 window(s) - [[1506890, 1606890], [1709515, 1904775], [4083139, 4183176]]
supported by 2 approach(es):
	2 candidate loci - [1759515, 1854775]
	1 window(s) - [[1709515, 1904775]]
supported by 3 approach(es):
	2 candidate loci - [1759515, 1854775]
	1 window(s) - [[1709515, 1904775]]

### scaffold_45 ###

supported by 1 approach(es):
	14 candidate loci - [223763, 306238, 357462, 357463, 358748, 363829, 370210, 384907, 384976, 393140, 406041, 406191, 472413, 472453]
	1 window(s) - [[173763, 522453]]
supported by 2 approach(es):
	2 candidate loci - [357462, 357463]
	1 window(s) - [[307462, 407463]]
supporte

Define some functions for plotting.

In [298]:


def print_Fst(filepath, filehandle, xmin, xmax, targets_by_length, targets, min_support, windows=[], support_colors=[], xticks=[], xlabels=[]):
    """write R commands to print Fst, etc"""
    
    OUT=filehandle
    p=filepath
    
    print "\n### %s ###\n" %p
    OUT.write("\n# read files\n")
    OUT.write("%s <- read.delim(file = '%s', header = T, sep = \"\\t\")\n" %(p.replace("-","_"), p))
    OUT.write("\npop = %s\n" %p.replace("-","_"))

    print "preparing plot"
    OUT.write("plot(%s, 0, axes=T, cex=0.5, ylab = \"\", xlab = \"\", ylim = c(0,1), xlim = c(%s/1000,%s/1000), yaxt = 'n', xaxt = 'n', col = 'white')\n" %(xmin, xmin, xmax ))
    OUT.write("axis(side = 2, at = c(0,0.5,1), labels = T, las=1, cex.axis=0.8)\n")
    if xticks and xlabels:
#    if files_to_process.index(p) == len(files_to_process)-1:
        OUT.write("axis(side = 1, at = c(%s), labels = c(%s), las=3, cex.axis=0.8)\n" %(",".join(xticks), ",".join(xlabels)))

    print "printing ackground color and supported windows if applicable"
    c=0
    for t in list(reversed(sorted(targets_by_length.keys()))): #[:3]:
        c+=1
        for scf in targets_by_length[t]:  
            print scf

            OUT.write("scf <- '%s'\n" %scf)
            OUT.write("scf\n")
            offset = float(targets[scf][3])/1000
            print "offset: %s" %offset
            
            if c % 2 == 0:
                OUT.write("rect(%s/1000+%s,0,%s/1000+%s,1, col = 'grey85', border = 'NA')\n" %(targets[scf][1], str(offset), targets[scf][2], str(offset)))
            else:
                OUT.write("rect(%s/1000+%s,0,%s/1000+%s,1, col = 'white', border = 'NA')\n" %(targets[scf][1], str(offset), targets[scf][2], str(offset)))
        
            for i in range(min_support, len(windows[scf])+1):
                print "supported by %i" %i
            
                for j in range(len(windows[scf][i])):
                    print "window %i: %s" %(j+1, windows[scf][i][j])
                    OUT.write("rect(%s/1000+%s,0,%s/1000+%s,1, col = '%s', border = '%s', lwd = '0.5')\n" %(windows[scf][i][j][0], str(offset), windows[scf][i][j][1], str(offset), support_colors[i], support_colors[i]))

    print "plotting data"
    for t in list(reversed(sorted(targets_by_length.keys()))): #[:3]:
        for scf in targets_by_length[t]:  
            print scf
            OUT.write("scf <- '%s'\n" %scf)
#            OUT.write("scf\n")
            offset = float(targets[scf][3])/1000
            print "offset: %s" %offset

            OUT.write("sub <- subset(pop, pop$Chr == scf)\n")
            OUT.write("lines(sub$BP/1000+%s, sub$Smoothed.AMOVA.Fst, type='b', lty=1, pch=20, cex = 0.8, lwd = 0.6)\n" %str(offset))
            OUT.write("points(sub$BP/1000+%s, sub$Smoothed.AMOVA.Fst, cex=0.8, pch=20, ylab = \"\", xlab = \"\", yaxt = 'n', xaxt = 'n')\n" %(str(offset)) )

    OUT.write("abline(h=mean(pop$AMOVA.Fst), lty=5, lwd=1)\n")
    OUT.write("abline(h=0.5, lty=3, lwd=0.2)\n")
    OUT.write("abline(h=0, lty=1, lwd=0.2)\n")
    OUT.write("abline(h=1, lty=1, lwd=0.2)\n")
    

In [306]:
import numpy as np
import os

#write R script

figure_prefix='sliding_window_joined-linetype-b-min-1'
fst_files = [x+".tsv" for x in pairs]

number_of_panels_vertical = 6
number_of_panels_horizontal = 1
wd = os.path.abspath(".") #working directory
#category = 'Eye' #support category to highlight
support_colors=["0","#d4fbddff", "#a6f7baff", "#16e74dff"]
min_support_for_print = 1


OUT=open(figure_prefix+'.R','w')
OUT.write("")

#set up dimensions of plot
matr=",".join([ str(x) for x in range(1,(number_of_panels_horizontal * number_of_panels_vertical)+1) ])
widths="1" #,".join(width)
heigths=",".join( [ str(1) for x in range(number_of_panels_horizontal * number_of_panels_vertical)])
#print matr
#print widths
#print heigths

OUT.write("#record the default parameters for plotting\ndefault_par <- par()\n\n")

OUT.write("\n# set working directory\nsetwd(dir = '%s')\n" %wd)
#OUT.write("\n# read files\n")
#for p in panel_prefix:
#    OUT.write("%s <- read.delim(file = '%s', header = T, sep = \"\\t\")\n" %(p.replace("-","_"), p))
    
OUT.write("\nsvg(filename = '%s.svg')\n" %figure_prefix)
OUT.write("mat <- matrix(c(%s), %i)\n" %(matr, number_of_panels_vertical))
OUT.write("layout(mat, widths=c(%s), heights=c(%s))\n" %(widths, heigths))
OUT.write("par(mar=c(0.2, 0, 0, 0), oma = c(3,3,1,1))\n")


#determine global minimum and maximum on the x axis of the plot
global_minimum = int(int(targets[targets_by_length[list(reversed(sorted(targets_by_length.keys())))[0]][0]][1]) + int(targets[targets_by_length[list(reversed(sorted(targets_by_length.keys())))[0]][0]][-2]))
print "start: %s" %global_minimum
global_maximum = int(int(targets[targets_by_length[list(reversed(sorted(targets_by_length.keys())))[-1]][0]][2]) + int(targets[targets_by_length[list(reversed(sorted(targets_by_length.keys())))[-1]][0]][-2]))
print "end: %s" %global_maximum


#determine positions for xticks and xlabels
ticks=[]
labels=[]
for t in list(reversed(sorted(targets_by_length.keys()))): #[:3]:
    for scf in targets_by_length[t]:
        print scf
        offset = float(targets[scf][3])/1000
        print offset
        labels.append(str(scf.split("_")[1]))
        print "start: %s" %str(float(targets[scf][1])/1000)
        print "end: %s" %str(float(targets[scf][2])/1000)
        ticks.append(str((float(targets[scf][2])/1000+offset+float(targets[scf][1])/1000+offset)/2))
        print ticks[-1]
#print ticks
#print labels

#print data into the panels

for f in fst_files[:-1]:
    print_Fst(filepath=f, filehandle=OUT, xmin=global_minimum, xmax=global_maximum, 
        targets_by_length=targets_by_length, targets=targets, min_support=min_support_for_print, 
        windows=windows, support_colors=["0","#d4fbddff", "#a6f7baff", "#16e74dff"])

print_Fst(filepath=fst_files[-1], filehandle=OUT, xmin=global_minimum, xmax=global_maximum, 
        targets_by_length=targets_by_length, targets=targets, min_support=min_support_for_print, 
        windows=windows, support_colors=["0","#d4fbddff", "#a6f7baff", "#16e74dff"], xticks=ticks, xlabels=labels)

    
OUT.write("\ndev.off()\n")
OUT.write("\n#reset to default\npar(default_par)")
OUT.close()


start: 10019
end: 19021293
scaffold_29
0.0
start: 10.019
end: 6366.611
3188.315
scaffold_45
6356.592
start: 44.68
end: 4415.838
8586.851
scaffold_81
10727.75
start: 43.414
end: 3030.239
12264.5765
scaffold_114
13714.575
start: 39.73
end: 2353.347
14911.1135
scaffold_174
16028.192
start: 12.998
end: 1293.689
16681.5355
scaffold_197
17308.883
start: 29.497
end: 952.411
17799.837
scaffold_219
18231.797
start: 0.805
end: 789.496
18626.9475

### Di_1-Di_2.tsv ###

preparing plot
printing ackground color and supported windows if applicable
scaffold_29
offset: 0.0
supported by 1
window 1: [1506890, 1606890]
window 2: [1709515, 1904775]
window 3: [4083139, 4183176]
supported by 2
window 1: [1709515, 1904775]
supported by 3
window 1: [1709515, 1904775]
scaffold_45
offset: 6356.592
supported by 1
window 1: [173763, 522453]
supported by 2
window 1: [307462, 407463]
supported by 3
window 1: [307462, 407463]
scaffold_81
offset: 10727.75
supported by 1
window 1: [318634, 500718]
supported by 2
windo

In [307]:
%%bash
Rscript sliding_window_joined-linetype-b.R
Rscript sliding_window_joined-linetype-b-min-1.R

[1] "scaffold_29"
[1] "scaffold_45"
[1] "scaffold_81"
[1] "scaffold_114"
[1] "scaffold_174"
[1] "scaffold_197"
[1] "scaffold_219"
[1] "scaffold_29"
[1] "scaffold_45"
[1] "scaffold_81"
[1] "scaffold_114"
[1] "scaffold_174"
[1] "scaffold_197"
[1] "scaffold_219"
[1] "scaffold_29"
[1] "scaffold_45"
[1] "scaffold_81"
[1] "scaffold_114"
[1] "scaffold_174"
[1] "scaffold_197"
[1] "scaffold_219"
[1] "scaffold_29"
[1] "scaffold_45"
[1] "scaffold_81"
[1] "scaffold_114"
[1] "scaffold_174"
[1] "scaffold_197"
[1] "scaffold_219"
[1] "scaffold_29"
[1] "scaffold_45"
[1] "scaffold_81"
[1] "scaffold_114"
[1] "scaffold_174"
[1] "scaffold_197"
[1] "scaffold_219"
[1] "scaffold_29"
[1] "scaffold_45"
[1] "scaffold_81"
[1] "scaffold_114"
[1] "scaffold_174"
[1] "scaffold_197"
[1] "scaffold_219"
pdf 
  2 
[1] "scaffold_29"
[1] "scaffold_45"
[1] "scaffold_81"
[1] "scaffold_114"
[1] "scaffold_174"
[1] "scaffold_197"
[1] "scaffold_219"
[1] "scaffold_29"
[1] "scaffold_45"
[1] "scaffold_81"
[1] "scaffold_114"
[1] "sc

Warning messages:
1: In par(default_par) : graphical parameter "cin" cannot be set
2: In par(default_par) : graphical parameter "cra" cannot be set
3: In par(default_par) : graphical parameter "csi" cannot be set
4: In par(default_par) : graphical parameter "cxy" cannot be set
5: In par(default_par) : graphical parameter "din" cannot be set
6: In par(default_par) : graphical parameter "page" cannot be set
Warning messages:
1: In par(default_par) : graphical parameter "cin" cannot be set
2: In par(default_par) : graphical parameter "cra" cannot be set
3: In par(default_par) : graphical parameter "csi" cannot be set
4: In par(default_par) : graphical parameter "cxy" cannot be set
5: In par(default_par) : graphical parameter "din" cannot be set
6: In par(default_par) : graphical parameter "page" cannot be set


In [11]:
cd ..

/media/chrishah/STORAGE/RAD/popgen/sliding_window_plots/DIPLOTAXODON_FOR_PAPER


#EYE morpholy

In [12]:
!mkdir eye

In [254]:
cd eye

/media/chrishah/STORAGE/RAD/popgen/sliding_window_plots/DIPLOTAXODON_FOR_PAPER/eye


Fetch the most signficicant list of candidates.

In [14]:
!ln -s /media/chrishah/STORAGE/RAD/popgen/Bayenv/Diplotaxodon/4_populations/M_zebra-BWA-8MM-stacks_m5_n5_r_0.8_p4_ONLY_ONE_SNP/ANALYSES_FOR_DIPLOTAXODON_PAPER_EXCLUDE_SINGLETONS/RANK_STATISTIC_2/EYE_regions.tsv .

In [52]:
!head EYE_regions.tsv


scaffold_12	3799847	6728
scaffold_148	543741	9559
scaffold_159	594470	11119
scaffold_197	125383	16065
scaffold_197	143069	16067
scaffold_197	150472	16069
scaffold_215	576905	19268
scaffold_39	1771937	31651


Fetch tables for the relevant Bayenv results.

In [16]:
!ln -s /media/chrishah/STORAGE/RAD/popgen/Bayenv/Diplotaxodon/4_populations/M_zebra-BWA-8MM-stacks_m5_n5_r_0.8_p4_ONLY_ONE_SNP/ANALYSES_FOR_DIPLOTAXODON_PAPER_EXCLUDE_SINGLETONS/RANK_STATISTIC_2/Diplo_1M_smoothed_V_EY_D_p_TL.txt .

In [17]:
!head Diplo_1M_smoothed_V_EY_D_p_TL.txt

chrom	bp	SNPID	std_rank	avg_rank_rel	avg_rank_rel_smoothed	avg_rank_rel_smoothed_p
scaffold_110	19064	3392	2113.46640617	0.611106397421	0.597815484148	0.234
scaffold_110	95096	3476	2628.00158438	0.546084337349	0.565868125823	0.3031
scaffold_110	95175	3475	2491.08418364	0.580154420499	0.575486997584	0.2837
scaffold_110	114547	3341	2168.34927768	0.631079246564	0.589864402219	0.2446
scaffold_110	234239	3412	3073.70548361	0.372984897336	0.580810969333	0.2728
scaffold_110	274029	3419	1212.20355036	0.814614797217	0.740481107386	0.0538
scaffold_110	284607	3420	665.711384535	0.90646105549	0.773929248539	0.0245
scaffold_110	452484	3439	1362.37131869	0.851747836416	0.851762923928	0.0007
scaffold_110	452630	3438	1087.69417117	0.875640590531	0.859834725637	0.0003


In [18]:
!ln -s /media/chrishah/STORAGE/RAD/popgen/Bayenv/Diplotaxodon/4_populations/M_zebra-BWA-8MM-stacks_m5_n5_r_0.8_p4_ONLY_ONE_SNP/ANALYSES_FOR_DIPLOTAXODON_PAPER_EXCLUDE_SINGLETONS/RANK_STATISTIC_2/Diplo_1M_V_EY_D_p_TL.txt .

In [19]:
!ln -s /media/chrishah/STORAGE/RAD/popgen/Bayenv/Diplotaxodon/4_populations/M_zebra-BWA-8MM-stacks_m5_n5_r_0.8_p4_ONLY_ONE_SNP/ANALYSES_FOR_DIPLOTAXODON_PAPER_EXCLUDE_SINGLETONS/RANK_STATISTIC_2/Diplo_1M_smoothed_H_Ey_D_p_TL.txt .

In [20]:
!head Diplo_1M_smoothed_H_Ey_D_p_TL.txt

chrom	bp	SNPID	std_rank	avg_rank_rel	avg_rank_rel_smoothed	avg_rank_rel_smoothed_p
scaffold_110	19064	3392	2022.60313149	0.624482436789	0.603153915122	0.2282
scaffold_110	95096	3476	2550.84432296	0.545460716104	0.56699833304	0.2929
scaffold_110	95175	3475	2494.38826168	0.576107245885	0.575650646525	0.2774
scaffold_110	114547	3341	2097.18469859	0.629916850501	0.590842499273	0.2397
scaffold_110	234239	3412	3036.15453288	0.374092143221	0.580256178975	0.2772
scaffold_110	274029	3419	1240.87408205	0.816668080774	0.740595879156	0.049
scaffold_110	284607	3420	713.484120356	0.900313931783	0.771057634312	0.0259
scaffold_110	452484	3439	1263.45095967	0.862154250806	0.85875931522	0.0005
scaffold_110	452630	3438	971.741914039	0.876976921772	0.863766927999	0.0004


In [21]:
!ln -s /media/chrishah/STORAGE/RAD/popgen/Bayenv/Diplotaxodon/4_populations/M_zebra-BWA-8MM-stacks_m5_n5_r_0.8_p4_ONLY_ONE_SNP/ANALYSES_FOR_DIPLOTAXODON_PAPER_EXCLUDE_SINGLETONS/RANK_STATISTIC_2/Diplo_1M_H_Ey_D_p_TL.txt .

Add column indicating the level of significance.

 - level 3: arr >= 0.95 & p < 0.001
 - level 2: arr >= 0.95
 - level 1: p < 0.05
 - level 0: the rest
 
 

In [262]:
#produce list of scaffold ids that contains SNPs that fullfill the most stringent criteria

candidate_scfs = [] #list of scaffolds that contain highly significant loci
top_95 = []

FH = open('Diplo_1M_smoothed_V_EY_D_p_TL.txt','r')
OUT = open('Diplo_1M_smoothed_V_EY_D_p_TL-incl_support.txt','w')

OUT.write(FH.readline().strip()+"\tsupport\n")
for l in FH:
    if float(l.strip().split("\t")[4]) >= 0.95:
        top_95.append(l.split("\t")[0])
        if float(l.strip().split("\t")[6]) <= 0.001:
            OUT.write(l.strip()+"\t3\n")
            candidate_scfs.append(l.split("\t")[0]) ##record the scf id
        else:
            OUT.write(l.strip()+"\t2\n")
    elif float(l.strip().split("\t")[6]) <= 0.05:
            OUT.write(l.strip()+"\t1\n")
    else:
        OUT.write(l.strip()+"\t0\n")
    

In [263]:


FH = open('Diplo_1M_smoothed_H_Ey_D_p_TL.txt','r')
OUT = open('Diplo_1M_smoothed_H_Ey_D_p_TL-incl_support.txt','w')

OUT.write(FH.readline().strip()+"\tsupport\n")
for l in FH:
    if float(l.strip().split("\t")[4]) >= 0.95:
        top_95.append(l.split("\t")[0])
        if float(l.strip().split("\t")[6]) <= 0.001:
            OUT.write(l.strip()+"\t3\n")
            candidate_scfs.append(l.split("\t")[0]) ##record the scf id
        else:
            OUT.write(l.strip()+"\t2\n")
    elif float(l.strip().split("\t")[6]) <= 0.05:
            OUT.write(l.strip()+"\t1\n")
    else:
        OUT.write(l.strip()+"\t0\n")
    

In [257]:
##non redundant list of scaffolds that contain highly significant candidate SNPs

candidate_scfs = list(set(candidate_scfs))
print candidate_scfs


['scaffold_159', 'scaffold_12', 'scaffold_215', 'scaffold_148', 'scaffold_39', 'scaffold_197']


The next cell goes through the pairwise Fst files from populations and records the Fst candidate support and some other metrics of the scaffolds of interest.

In [258]:
#these are the scaffolds that will be processed
print "Candidates scaffolds: %s" %sorted(candidate_scfs)

targets = {}

#records all SNP coordinates for the scaffolds of interest
for p in pairs:
    print "processint %s" %p
    FH = open("../pairwise_Fst/"+p+".tsv", 'r')
    for l in FH:
#        print l.strip()
        if l.split("\t")[4] in candidate_scfs:
            if not l.split("\t")[4] in targets:
                targets[l.split("\t")[4]] = []
            targets[l.split("\t")[4]].append(int(l.split("\t")[5]))
            
    FH.close()

for k in targets.keys():
        targets[k] = sorted(list(set(targets[k])))                 

#print targets    
print "DONE"
                    
targets_by_length = {}
print targets.keys()
for t in sorted(targets):
    if not len(set(targets[t])) > 1:
        print "excluding %s" %t
        continue
#    print t,len(targets[t])
#    print sorted(targets[t])[0],sorted(targets[t])[-1]
    length=sorted(targets[t])[-1] - sorted(targets[t])[0]
    targets[t]=[str(length), str(sorted(targets[t])[0]), str(sorted(targets[t])[-1])]
    if not length in targets_by_length:
        targets_by_length[length] = []
    targets_by_length[length].append(t)
#    print targets[t]

longest=int(sorted(targets_by_length)[-1])
count=0


##
cummulative = 0
for t in reversed(sorted(targets_by_length.keys())):
#    print t
    for scf in targets_by_length[t]:
#        print scf
        
        targets[scf].append(str(cummulative))
        factor="%.2f" %(float(targets[scf][0])/longest)
        targets[scf].append(factor)
        print "%s\t%s" %(scf, "\t".join(targets[scf]))#, float(targets[scf][0])/longest)
        count+=1
        cummulative+=int(targets[scf][0])

print "Total number of scaffolds to print: %s" %count

Candidates scaffolds: ['scaffold_12', 'scaffold_148', 'scaffold_159', 'scaffold_197', 'scaffold_215', 'scaffold_39']
processint Di_1-Di_2
processint Di_1-Di_4
processint Di_1-Di_5
processint Di_2-Di_4
processint Di_2-Di_5
processint Di_4-Di_5
DONE
['scaffold_159', 'scaffold_12', 'scaffold_215', 'scaffold_148', 'scaffold_39', 'scaffold_197']
scaffold_12	8622592	71177	8693769	0	1.00
scaffold_39	4791200	80328	4871528	8622592	0.56
scaffold_148	1653445	43527	1696972	13413792	0.19
scaffold_159	1613400	23794	1637194	15067237	0.19
scaffold_197	922914	29497	952411	16680637	0.11
scaffold_215	715698	135232	850930	17603551	0.08
Total number of scaffolds to print: 6


The next identifies windows of interest and the respective level of support for Fst and Eye morphology candidates on all candidate scaffolds.

In [259]:
regions = {}
windows = {}
dist=50000 #window size in bp (up and downstream of SNP)

First Fst.

In [58]:
#create symbolic links to Fst data files
!ln -s ../pairwise_Fst/Di_* .

ln: failed to create symbolic link ‘./Di_1-Di_2.tsv’: File exists
ln: failed to create symbolic link ‘./Di_1-Di_4.tsv’: File exists
ln: failed to create symbolic link ‘./Di_1-Di_5.tsv’: File exists
ln: failed to create symbolic link ‘./Di_2-Di_4.tsv’: File exists
ln: failed to create symbolic link ‘./Di_2-Di_5.tsv’: File exists
ln: failed to create symbolic link ‘./Di_4-Di_5.tsv’: File exists


In [260]:


#print candidates

regions['Fst'] = {}

#add placeholders for all candidate scaffolds
for c in candidate_scfs:
    regions['Fst'][c] = {}

#loop over the pairwise comparisons and record Fst candidate SNPs and their level of support
for p in pairs:
    out=[]
    ids=[]
    print "processint %s" %p
    FH = open(p+".tsv", 'r')
    FH.readline()
    for l in FH:
        if l.split("\t")[4] in candidate_scfs: #if the scaffold is one of the candidates
            #extract the number of approaches supporting a given SNP as candidate locus
#            print "%s is a candidate tag" %l.split("\t")[1]
            support = l.strip().split("\t")[-1]
            if support > 0:
#                print "\n%s\t%s\t%s\t%s" %(l.split("\t")[1], l.split("\t")[4], l.split("\t")[5], support)

                for s in range(1,int(support)+1):
                    if not s in regions['Fst'][l.split("\t")[4]]:
                        regions['Fst'][l.split("\t")[4]][int(s)] = []
#                print regions
                for s in range(1,int(support)+1):
#                    print s
                    regions['Fst'][l.split("\t")[4]][s].append(int(l.split("\t")[5]))
#                print regions

        
    FH.close()   
    print "Done\n"

#identify candidate windows
windows['Fst'] = {}
for r in sorted(targets):#regions):
    print r
    if not r in windows['Fst']:
        windows['Fst'][r]={}
    for s in regions['Fst'][r]:
        if not s in windows['Fst'][r]:
            windows['Fst'][r][s]=[]
#        print "\tsupport: %s" %s
        regions['Fst'][r][s]=sorted(list(set(regions['Fst'][r][s])))

        number = len(regions['Fst'][r][s])
#        print "\t\t %i candidate loci - %s" %(number, str(regions[r][s]))
        lower = regions['Fst'][r][s][0]-dist
        if lower < int(targets[r][1]):
            lower = int(targets[r][1])
        upper = regions['Fst'][r][s][0]+dist
        if upper > int(targets[r][2]):
            upper = int(targets[r][2])
        for i in range(len(regions['Fst'][r][s])-1):
#            print "\t\t%i\t%i" %(i,regions[r][s][i])
            if regions['Fst'][r][s][i] + dist >= regions['Fst'][r][s][i+1]-dist:
#                print "\t\t\t%i is in range - making %i the upper boundery" %(regions[r][s][i+1], regions[r][s][i+1]+dist)
                upper = regions['Fst'][r][s][i+1] + dist
            else:
#                print "\t\t\t%i is NOT in range" %regions[r][s][i+1]
                windows['Fst'][r][s].append([lower, upper])
                lower = regions['Fst'][r][s][i+1]-dist
                if lower < int(targets[r][1]):
                    lower = int(targets[r][1])
                upper = regions['Fst'][r][s][i+1]+dist
                if upper > int(targets[r][2]):
                    upper = int(targets[r][2])
        windows['Fst'][r][s].append([lower, upper])
#        print "\t\t %i window(s): %s" %(len(windows[r][s]),windows[r][s])
                

                
for t in reversed(sorted(targets_by_length.keys())):
#    print t
    for scf in targets_by_length[t]:
        print "\n### %s ###\n" %scf
        for i in range(1,len(windows['Fst'][scf])+1):
            print "supported by %i approach(es):" %i
            print "\t%i candidate loci - %s" %(len(regions['Fst'][scf][i]), regions['Fst'][scf][i])
            print "\t%i window(s) - %s" %(len(windows['Fst'][scf][i]), windows['Fst'][scf][i])

processint Di_1-Di_2
Done

processint Di_1-Di_4
Done

processint Di_1-Di_5
Done

processint Di_2-Di_4
Done

processint Di_2-Di_5
Done

processint Di_4-Di_5
Done

scaffold_12
scaffold_148
scaffold_159
scaffold_197
scaffold_215
scaffold_39

### scaffold_12 ###

supported by 1 approach(es):
	1 candidate loci - [3799846]
	1 window(s) - [[3749846, 3849846]]
supported by 2 approach(es):
	1 candidate loci - [3799846]
	1 window(s) - [[3749846, 3849846]]

### scaffold_39 ###

supported by 1 approach(es):
	5 candidate loci - [1141783, 1771896, 1771936, 1814919, 1815034]
	2 window(s) - [[1091783, 1191783], [1721896, 1865034]]
supported by 2 approach(es):
	3 candidate loci - [1771896, 1771936, 1814919]
	1 window(s) - [[1721896, 1864919]]

### scaffold_148 ###


### scaffold_159 ###


### scaffold_197 ###

supported by 1 approach(es):
	12 candidate loci - [125382, 143068, 150471, 205073, 205100, 212069, 261800, 678871, 680501, 741817, 741818, 792059]
	2 window(s) - [[75382, 311800], [628871, 842059

The next identifies and adds windows of interest and the respective level of support for Eye candidates on all candidate scaffolds.

In [261]:
to_process = ['Diplo_1M_smoothed_V_EY_D_p_TL', 'Diplo_1M_smoothed_H_Ey_D_p_TL']

#print candidates

regions['Eye'] = {}

#add placeholders for all candidate scaffolds
for c in candidate_scfs:
    regions['Eye'][c] = {}

#loop over the pairwise comparisons and record Eye candidate SNPs and their level of support
for p in to_process:
    out=[]
    ids=[]
    print "processint %s" %p
    FH = open(p+"-incl_support.txt", 'r')
    FH.readline()
    for l in FH:
        if l.split("\t")[0] in candidate_scfs: #if the scaffold is one of the candidates
            #extract the number of approaches supporting a given SNP as candidate locus
#            print "%s is a candidate tag" %l.split("\t")[1]
            support = l.strip().split("\t")[-1]
            if support > 0:
#                print "\n%s\t%s\t%s\t%s" %(l.split("\t")[1], l.split("\t")[4], l.split("\t")[5], support)

                for s in range(1,int(support)+1):
                    if not s in regions['Eye'][l.split("\t")[0]]:
                        regions['Eye'][l.split("\t")[0]][int(s)] = []
#                print regions
                for s in range(1,int(support)+1):
#                    print s
                    regions['Eye'][l.split("\t")[0]][s].append(int(l.split("\t")[1]))
#                print regions

        
    FH.close()   
    print "Done\n"

#identify candidate windows
windows['Eye'] = {}
for r in sorted(targets):#regions):
    print r
    if not r in windows['Eye']:
        windows['Eye'][r]={}
    for s in regions['Eye'][r]:
        if not s in windows['Eye'][r]:
            windows['Eye'][r][s]=[]
#        print "\tsupport: %s" %s
        regions['Eye'][r][s]=sorted(list(set(regions['Eye'][r][s])))

        number = len(regions['Eye'][r][s])
#        print "\t\t %i candidate loci - %s" %(number, str(regions[r][s]))
        lower = regions['Eye'][r][s][0]-dist
        if lower < int(targets[r][1]):
            lower = int(targets[r][1])
        upper = regions['Eye'][r][s][0]+dist
        if upper > int(targets[r][2]):
            upper = int(targets[r][2])
        for i in range(len(regions['Eye'][r][s])-1):
#            print "\t\t%i\t%i" %(i,regions[r][s][i])
            if regions['Eye'][r][s][i] + dist >= regions['Eye'][r][s][i+1]-dist:
#                print "\t\t\t%i is in range - making %i the upper boundery" %(regions[r][s][i+1], regions[r][s][i+1]+dist)
                upper = regions['Eye'][r][s][i+1] + dist
            else:
#                print "\t\t\t%i is NOT in range" %regions[r][s][i+1]
                windows['Eye'][r][s].append([lower, upper])
                lower = regions['Eye'][r][s][i+1]-dist
                if lower < int(targets[r][1]):
                    lower = int(targets[r][1])
                upper = regions['Eye'][r][s][i+1]+dist
                if upper > int(targets[r][2]):
                    upper = int(targets[r][2])
        windows['Eye'][r][s].append([lower, upper])
#        print "\t\t %i window(s): %s" %(len(windows[r][s]),windows[r][s])
                

                
for t in reversed(sorted(targets_by_length.keys())):
#    print t
    for scf in targets_by_length[t]:
        print "\n### %s ###\n" %scf
        for i in range(1,len(windows['Eye'][scf])+1):
            print "level of support: %i" %i
            print "\t%i candidate loci - %s" %(len(regions['Eye'][scf][i]), regions['Eye'][scf][i])
            print "\t%i window(s) - %s" %(len(windows['Eye'][scf][i]), windows['Eye'][scf][i])

processint Diplo_1M_smoothed_V_EY_D_p_TL
Done

processint Diplo_1M_smoothed_H_Ey_D_p_TL
Done

scaffold_12
scaffold_148
scaffold_159
scaffold_197
scaffold_215
scaffold_39

### scaffold_12 ###

level of support: 1
	9 candidate loci - [2401139, 2435698, 2985917, 3799847, 3885790, 3913836, 3934807, 4201555, 7678845]
	5 window(s) - [[2351139, 2485698], [2935917, 3035917], [3749847, 3984807], [4151555, 4251555], [7628845, 7728845]]
level of support: 2
	2 candidate loci - [3799847, 7678845]
	2 window(s) - [[3749847, 3849847], [7628845, 7728845]]
level of support: 3
	1 candidate loci - [3799847]
	1 window(s) - [[3749847, 3849847]]

### scaffold_39 ###

level of support: 1
	10 candidate loci - [1141784, 1277431, 1673547, 1688054, 1771937, 1814976, 1815035, 2072806, 2127729, 2127806]
	4 window(s) - [[1091784, 1191784], [1227431, 1327431], [1623547, 1865035], [2022806, 2177806]]
level of support: 2
	2 candidate loci - [1277431, 1771937]
	2 window(s) - [[1227431, 1327431], [1721937, 1821937]]
leve

Summarize the candidate windows.

In [61]:
categories = ['Fst','Eye']

for t in reversed(sorted(targets_by_length.keys())):
#    print t
    for scf in targets_by_length[t]:
        print "\n### %s ###\n" %scf
        for c in categories:
            print "\n## Candidate category: %s ##\n" %c
            for i in range(1,len(windows[c][scf])+1):
                print "level of support: %i" %i
                print "\t%i candidate loci - %s" %(len(regions[c][scf][i]), regions[c][scf][i])
                print "\t%i window(s) - %s" %(len(windows[c][scf][i]), windows[c][scf][i])


### scaffold_12 ###


## Candidate category: Fst ##

level of support: 1
	1 candidate loci - [3799846]
	1 window(s) - [[3749846, 3849846]]
level of support: 2
	1 candidate loci - [3799846]
	1 window(s) - [[3749846, 3849846]]

## Candidate category: Eye ##

level of support: 1
	9 candidate loci - [2401139, 2435698, 2985917, 3799847, 3885790, 3913836, 3934807, 4201555, 7678845]
	5 window(s) - [[2351139, 2485698], [2935917, 3035917], [3749847, 3984807], [4151555, 4251555], [7628845, 7728845]]
level of support: 2
	2 candidate loci - [3799847, 7678845]
	2 window(s) - [[3749847, 3849847], [7628845, 7728845]]
level of support: 3
	1 candidate loci - [3799847]
	1 window(s) - [[3749847, 3849847]]

### scaffold_39 ###


## Candidate category: Fst ##

level of support: 1
	5 candidate loci - [1141783, 1771896, 1771936, 1814919, 1815034]
	2 window(s) - [[1091783, 1191783], [1721896, 1865034]]
level of support: 2
	3 candidate loci - [1771896, 1771936, 1814919]
	1 window(s) - [[1721896, 1864919]]

##

Print results

In [62]:
#These are the scaffolds to be printed
for t in reversed(sorted(targets_by_length.keys())):
    print targets_by_length[t],targets[targets_by_length[t][0]]

['scaffold_12'] ['8622592', '71177', '8693769', '0', '1.00']
['scaffold_39'] ['4791200', '80328', '4871528', '8622592', '0.56']
['scaffold_148'] ['1653445', '43527', '1696972', '13413792', '0.19']
['scaffold_159'] ['1613400', '23794', '1637194', '15067237', '0.19']
['scaffold_197'] ['922914', '29497', '952411', '16680637', '0.11']
['scaffold_215'] ['715698', '135232', '850930', '17603551', '0.08']


Some functions for plotting.

In [133]:
def max_sd(number, maxi):
    import numpy as np
    
    li=[ 1 for x in range(number/2)] #half the observations are 1
    li.extend([ maxi for x in range(number/2)]) #half the observations are the maximum
    sd_max = np.std(li)
    
    return sd_max

def calculate_ARR_global_weighted_average(filepath, sd_max):
    import numpy as np
    
    IN = open(filepath, 'r')
    to_average=[]

    IN.readline()
    for l in IN:
        weight = (1-float(l.strip().split("\t")[6]) / sd_max)
        weighted = float(l.strip().split("\t")[9]) * weight
#        print l.strip().split("\t")[6], l.strip().split("\t")[9], weight,  weighted
        to_average.append(weighted)
    
    #print to_average
    return np.average(to_average)
    

def print_ARR(filepath, filehandle, xmin, xmax, targets_by_length, targets, min_support, 
              windows=[], support_colors=[], xticks=[], xlabels=[], global_avg=0, sd_max=0):
    """write R commands to print average relative ranks, etc"""
    
    if not sd_max:
        weight = "1"
    else:
        weight = "(1-(sub$std_rank/%s))" %sd_max
    
    OUT=filehandle
    p=filepath
    
    print "\n### %s ###\n" %p
    OUT.write("\n# read files\n")
    OUT.write("%s <- read.delim(file = '%s', header = T, sep = \"\\t\")\n" %(p.replace("-","_"), p))
    OUT.write("\npop = %s\n" %p.replace("-","_"))
    OUT.write("plot(%s, 0, axes=T, cex=0.5, ylab = \"\", xlab = \"\", ylim = c(0,1), xlim = c(%s/1000,%s/1000), yaxt = 'n', xaxt = 'n', col = 'white')\n" %(xmin, xmin, xmax ))
    OUT.write("axis(side = 2, at = c(0,0.5,1), labels = T, las=1, cex.axis=0.8)\n")
    if xticks and xlabels:
#    if files_to_process.index(p) == len(files_to_process)-1:
        OUT.write("axis(side = 1, at = c(%s), labels = c(%s), las=3, cex.axis=1)\n" %(",".join(xticks), ",".join(xlabels)))
    c=0
    for t in list(reversed(sorted(targets_by_length.keys()))): #[:3]:
        c+=1
        for scf in targets_by_length[t]:  
#            print scf

            OUT.write("scf <- '%s'\n" %scf)
            OUT.write("scf\n")
            offset = float(targets[scf][3])/1000
#            print "offset: %s" %offset
            
            if c % 2 == 0:
                OUT.write("rect(%s/1000+%s,0,%s/1000+%s,1, col = 'grey85', border = 'NA')\n" %(targets[scf][1], str(offset), targets[scf][2], str(offset)))
            else:
                OUT.write("rect(%s/1000+%s,0,%s/1000+%s,1, col = 'white', border = 'NA')\n" %(targets[scf][1], str(offset), targets[scf][2], str(offset)))
        
            for i in range(min_support, len(windows[scf])+1):
#                print "supported by %i" %i
            
                for j in range(len(windows[scf][i])):
#                    print "window %i: %s" %(j+1, windows[scf][i][j])
                    OUT.write("rect(%s/1000+%s,0,%s/1000+%s,1, col = '%s', border = '%s', lwd = '0.5')\n" %(windows[scf][i][j][0], str(offset), windows[scf][i][j][1], str(offset), support_colors[i], support_colors[i]))

    for t in list(reversed(sorted(targets_by_length.keys()))): #[:3]:
        for scf in targets_by_length[t]:  
#            print scf
            OUT.write("scf <- '%s'\n" %scf)
#            OUT.write("scf\n")
            offset = float(targets[scf][3])/1000
#            print "offset: %s" %offset

            OUT.write("sub <- subset(pop, pop$chrom == scf)\n")
            
            OUT.write("#points(sub$bp/1000+%s, sub$avg_rank_rel, col='grey',cex=0.8, pch=20, ylab = \"\", xlab = \"\", yaxt = 'n', xaxt = 'n')\n" %(str(offset)) )
            OUT.write("#points(sub$bp/1000+%s, sub$avg_rank_rel*%s, col='blue', cex=0.8, pch=20, ylab = \"\", xlab = \"\", yaxt = 'n', xaxt = 'n')\n" %(str(offset), weight) )
            OUT.write("lines(sub$bp/1000+%s, sub$avg_rank_rel_smoothed*%s, type='b', col='black', lty=1, pch=20, cex = 0.8, lwd = 1)\n" %(str(offset), weight) )
            OUT.write("#lines(sub$bp/1000+%s, sub$avg_rank_rel, type='b', col='red', lty=1, pch=20, cex = 0.8, lwd = 0.6)\n" %str(offset))
    if global_avg:
        OUT.write("abline(h=%s, lty=5, lwd=1)\n" %global_avg)
    else: #calculate from data in file - this may yield incorrect result if data in file is not complete
        OUT.write("abline(h=mean(pop$avg_rank_rel), lty=5, lwd=1)\n")
    OUT.write("abline(h=0.95, lty=3, lwd=0.2)\n")
    OUT.write("abline(h=0.5, lty=3, lwd=0.2)\n")
    OUT.write("abline(h=0, lty=1, lwd=0.2)\n")
    OUT.write("abline(h=1, lty=1, lwd=0.2)\n")


def print_Fst(filepath, filehandle, xmin, xmax, targets_by_length, targets, min_support, windows=[], support_colors=[], xticks=[], xlabels=[]):
    """write R commands to print Fst, etc"""
    
    OUT=filehandle
    p=filepath
    
    print "\n### %s ###\n" %p
    OUT.write("\n# read files\n")
    OUT.write("%s <- read.delim(file = '%s', header = T, sep = \"\\t\")\n" %(p.replace("-","_"), p))
    OUT.write("\npop = %s\n" %p.replace("-","_"))

    print "preparing plot"
    OUT.write("plot(%s, 0, axes=T, cex=0.5, ylab = \"\", xlab = \"\", ylim = c(0,1), xlim = c(%s/1000,%s/1000), yaxt = 'n', xaxt = 'n', col = 'white')\n" %(xmin, xmin, xmax ))
    OUT.write("axis(side = 2, at = c(0,0.5,1), labels = T, las=1, cex.axis=0.8)\n")
    if xticks and xlabels:
#    if files_to_process.index(p) == len(files_to_process)-1:
        OUT.write("axis(side = 1, at = c(%s), labels = c(%s), las=3, cex.axis=1)\n" %(",".join(xticks), ",".join(xlabels)))

    print "printing background color and supported windows if applicable"
    c=0
    for t in list(reversed(sorted(targets_by_length.keys()))): #[:3]:
        c+=1
        for scf in targets_by_length[t]:  
#            print scf

            OUT.write("scf <- '%s'\n" %scf)
#            OUT.write("scf\n")
            offset = float(targets[scf][3])/1000
#            print "offset: %s" %offset
            
            if c % 2 == 0:
                OUT.write("rect(%s/1000+%s,0,%s/1000+%s,1, col = 'grey85', border = 'NA')\n" %(targets[scf][1], str(offset), targets[scf][2], str(offset)))
            else:
                OUT.write("rect(%s/1000+%s,0,%s/1000+%s,1, col = 'white', border = 'NA')\n" %(targets[scf][1], str(offset), targets[scf][2], str(offset)))
        
            for i in range(min_support, len(windows[scf])+1):
#                print "supported by %i" %i
            
                for j in range(len(windows[scf][i])):
#                    print "window %i: %s" %(j+1, windows[scf][i][j])
                    OUT.write("rect(%s/1000+%s,0,%s/1000+%s,1, col = '%s', border = '%s', lwd = '0.5')\n" %(windows[scf][i][j][0], str(offset), windows[scf][i][j][1], str(offset), support_colors[i], support_colors[i]))

    print "plotting data"
    for t in list(reversed(sorted(targets_by_length.keys()))): #[:3]:
        for scf in targets_by_length[t]:  
#            print scf
            OUT.write("scf <- '%s'\n" %scf)
#            OUT.write("scf\n")
            offset = float(targets[scf][3])/1000
#            print "offset: %s" %offset

            OUT.write("sub <- subset(pop, pop$Chr == scf)\n")
            OUT.write("lines(sub$BP/1000+%s, sub$Smoothed.AMOVA.Fst, type='b', lty=1, pch=20, cex = 0.8, lwd = 0.6)\n" %str(offset))
            OUT.write("points(sub$BP/1000+%s, sub$Smoothed.AMOVA.Fst, cex=0.8, pch=20, ylab = \"\", xlab = \"\", yaxt = 'n', xaxt = 'n')\n" %(str(offset)) )

    OUT.write("abline(h=mean(pop$AMOVA.Fst), lty=5, lwd=1)\n")
    OUT.write("abline(h=0.5, lty=3, lwd=0.2)\n")
    OUT.write("abline(h=0, lty=1, lwd=0.2)\n")
    OUT.write("abline(h=1, lty=1, lwd=0.2)\n")

    


In [134]:
import numpy as np
import os

##################################################
#calculate maxiumum possible standard deviation
sd_max = max_sd(number=20, maxi=11786)

#calculate global weighted average for ARR from the full dataset
global_average = calculate_ARR_global_weighted_average(filepath='Diplo_1M_V_EY_D_p_TL.txt', sd_max=sd_max)

#################################################


#write R script

figure_prefix='sliding_window_Eye_stringent'
fst_files = [x+".tsv" for x in pairs]
Eye_files = ['Diplo_1M_smoothed_V_EY_D_p_TL-incl_support.txt', 'Diplo_1M_smoothed_H_Ey_D_p_TL-incl_support.txt'] #prefix of the files to be processed

number_of_panels_vertical = 7
number_of_panels_horizontal = 1
wd = os.path.abspath(".") #working directory
#category = 'Eye' #support category to highlight
support_colors=["0","#d4fbddff", "#a6f7baff", "#16e74dff"]
min_support_for_print = 2


OUT=open(figure_prefix+'.R','w')
OUT.write("")

#set up dimensions of plot
matr=",".join([ str(x) for x in range(1,(number_of_panels_horizontal * number_of_panels_vertical)+1) ])
widths="1" #,".join(width)
heigths=",".join( [ str(1) for x in range(number_of_panels_horizontal * number_of_panels_vertical)])
#print matr
#print widths
#print heigths

OUT.write("#record the default parameters for plotting\ndefault_par <- par()\n\n")

OUT.write("\n# set working directory\nsetwd(dir = '%s')\n" %wd)
#OUT.write("\n# read files\n")
#for p in panel_prefix:
#    OUT.write("%s <- read.delim(file = '%s', header = T, sep = \"\\t\")\n" %(p.replace("-","_"), p))
    
OUT.write("\nsvg(filename = '%s.svg')\n" %figure_prefix)
OUT.write("mat <- matrix(c(%s), %i)\n" %(matr, number_of_panels_vertical))
OUT.write("layout(mat, widths=c(%s), heights=c(%s))\n" %(widths, heigths))
OUT.write("par(mar=c(0.2, 0, 0, 0), oma = c(3,3,1,1))\n")


#determine global minimum and maximum on the x axis of the plot
global_minimum = int(int(targets[targets_by_length[list(reversed(sorted(targets_by_length.keys())))[0]][0]][1]) + int(targets[targets_by_length[list(reversed(sorted(targets_by_length.keys())))[0]][0]][-2]))
print "start: %s" %global_minimum
global_maximum = int(int(targets[targets_by_length[list(reversed(sorted(targets_by_length.keys())))[-1]][0]][2]) + int(targets[targets_by_length[list(reversed(sorted(targets_by_length.keys())))[-1]][0]][-2]))
print "end: %s" %global_maximum


#determine positions for xticks and xlabels
ticks=[]
labels=[]
for t in list(reversed(sorted(targets_by_length.keys()))): #[:3]:
    for scf in targets_by_length[t]:
#        print scf
        offset = float(targets[scf][3])/1000
#        print offset
        labels.append(str(scf.split("_")[1]))
#        print "start: %s" %str(float(targets[scf][1])/1000)
#        print "end: %s" %str(float(targets[scf][2])/1000)
        ticks.append(str((float(targets[scf][2])/1000+offset+float(targets[scf][1])/1000+offset)/2))
#        print ticks[-1]
#print ticks
#print labels

#print data into the panels
for f in fst_files:
    print_Fst(filepath=f, filehandle=OUT, xmin=global_minimum, xmax=global_maximum, 
        targets_by_length=targets_by_length, targets=targets, min_support=min_support_for_print, 
        windows=windows['Eye'], support_colors=["0","#d4fbddff", "#a6f7baff", "#16e74dff"])

print_ARR(filepath=Eye_files[0], filehandle=OUT, xmin=global_minimum, xmax=global_maximum, 
    targets_by_length=targets_by_length, targets=targets, min_support=min_support_for_print, 
    windows=windows['Eye'], support_colors=["0","#d4fbddff", "#a6f7baff", "#16e74dff"], 
    xticks=ticks, xlabels=labels, global_avg=global_average, sd_max=sd_max)

OUT.write("\ndev.off()\n")
OUT.write("\n#reset to default\npar(default_par)")
OUT.close()


start: 71177
end: 18454481

### Di_1-Di_2.tsv ###

preparing plot
printing background color and supported windows if applicable
plotting data

### Di_1-Di_4.tsv ###

preparing plot
printing background color and supported windows if applicable
plotting data

### Di_1-Di_5.tsv ###

preparing plot
printing background color and supported windows if applicable
plotting data

### Di_2-Di_4.tsv ###

preparing plot
printing background color and supported windows if applicable
plotting data

### Di_2-Di_5.tsv ###

preparing plot
printing background color and supported windows if applicable
plotting data

### Di_4-Di_5.tsv ###

preparing plot
printing background color and supported windows if applicable
plotting data

### Diplo_1M_smoothed_V_EY_D_p_TL-incl_support.txt ###



In [135]:
!Rscript sliding_window_Eye_stringent.R

[1] "scaffold_12"
[1] "scaffold_39"
[1] "scaffold_148"
[1] "scaffold_159"
[1] "scaffold_197"
[1] "scaffold_215"
pdf 
  2 
Warning messages:
1: In par(default_par) : graphical parameter "cin" cannot be set
2: In par(default_par) : graphical parameter "cra" cannot be set
3: In par(default_par) : graphical parameter "csi" cannot be set
4: In par(default_par) : graphical parameter "cxy" cannot be set
5: In par(default_par) : graphical parameter "din" cannot be set
6: In par(default_par) : graphical parameter "page" cannot be set


In [136]:
def print_arrows(filehandle, xmin, xmax, targets_by_length, targets, support=0, windows=[], support_colors=[]):
    """write R commands to print arrows"""
    
    OUT=filehandle
    print "\n### print arrows ###\n"
    OUT.write("library(shape)\n")
    print "preparing plot"
    OUT.write("plot(%s, 0, axes=F, cex=0.5, ylab = \"\", xlab = \"\", ylim = c(0,1), xlim = c(%s/1000,%s/1000), yaxt = 'n', xaxt = 'n', col = 'white')\n" %(xmin, xmin, xmax ))

    print "print arrows"
    for scf in windows.keys(): #these are all scaffolds that contain a candidates SNP
        if len(windows[scf]) > 0:
#            print scf
            offset = float(targets[scf][3])/1000
#            print "offset: %s" %offset
#            for i in range(min_support, len(windows[scf])+1):
#                print "supported by %i" %i
            
            for j in range(len(windows[scf][support])):
#                print "window %i: %s" %(j+1, windows[scf][i][j])
                center = np.average([windows[scf][support][j][0], windows[scf][support][j][1]])
                OUT.write("#rect(%s/1000+%s,0,%s/1000+%s,1, col = '%s', border = '%s', lwd = '0.5')\n" %(windows[scf][support][j][0], str(offset), windows[scf][support][j][1], str(offset), support_colors[i], support_colors[i]))
                OUT.write("center <- %s/1000\n" %center)
                    
                OUT.write("Arrows(x0=center+%s, y0=1, x1=center+%s, y1=0, col='%s', arr.adj = 1, arr.width=0.5, arr.type='triangle')\n" %(offset, offset, support_colors[support]))

In [143]:
import numpy as np
import os

##################################################
#calculate maxiumum possible standard deviation
sd_max = max_sd(number=20, maxi=11786)

#calculate global weighted average for ARR from the full dataset
global_average = calculate_ARR_global_weighted_average(filepath='Diplo_1M_V_EY_D_p_TL.txt', sd_max=sd_max)

#################################################


#write R script

figure_prefix='sliding_window_Eye_stringent-arrows'
fst_files = [x+".tsv" for x in pairs]
Eye_files = ['Diplo_1M_smoothed_V_EY_D_p_TL-incl_support.txt', 'Diplo_1M_smoothed_H_Ey_D_p_TL-incl_support.txt'] #prefix of the files to be processed

number_of_panels_vertical = 8
number_of_panels_horizontal = 1
wd = os.path.abspath(".") #working directory
#category = 'Eye' #support category to highlight
support_colors=["0","#d4fbddff", "#a6f7baff", "#16e74dff"]
min_support_for_print = 2


OUT=open(figure_prefix+'.R','w')
OUT.write("")

#set up dimensions of plot
matr=",".join([ str(x) for x in range(1,(number_of_panels_horizontal * number_of_panels_vertical)+1) ])
widths="1" #,".join(width)


heigths = "0.2," #the height of the first panel
heigths+=",".join( [ str(1) for x in range(number_of_panels_horizontal * number_of_panels_vertical - 1)])

#print matr
#print widths
#print heigths

OUT.write("#record the default parameters for plotting\ndefault_par <- par()\n\n")

OUT.write("\n# set working directory\nsetwd(dir = '%s')\n" %wd)
#OUT.write("\n# read files\n")
#for p in panel_prefix:
#    OUT.write("%s <- read.delim(file = '%s', header = T, sep = \"\\t\")\n" %(p.replace("-","_"), p))
    
OUT.write("\nsvg(filename = '%s.svg')\n" %figure_prefix)
OUT.write("mat <- matrix(c(%s), %i)\n" %(matr, number_of_panels_vertical))
OUT.write("layout(mat, widths=c(%s), heights=c(%s))\n" %(widths, heigths))
OUT.write("par(mar=c(0.2, 0, 0, 0), oma = c(3,3,1,1))\n")


#determine global minimum and maximum on the x axis of the plot
global_minimum = int(int(targets[targets_by_length[list(reversed(sorted(targets_by_length.keys())))[0]][0]][1]) + int(targets[targets_by_length[list(reversed(sorted(targets_by_length.keys())))[0]][0]][-2]))
print "start: %s" %global_minimum
global_maximum = int(int(targets[targets_by_length[list(reversed(sorted(targets_by_length.keys())))[-1]][0]][2]) + int(targets[targets_by_length[list(reversed(sorted(targets_by_length.keys())))[-1]][0]][-2]))
print "end: %s" %global_maximum


#determine positions for xticks and xlabels
ticks=[]
labels=[]
for t in list(reversed(sorted(targets_by_length.keys()))): #[:3]:
    for scf in targets_by_length[t]:
        print scf
        offset = float(targets[scf][3])/1000
        print offset
        labels.append(str(scf.split("_")[1]))
        print "start: %s" %str(float(targets[scf][1])/1000)
        print "end: %s" %str(float(targets[scf][2])/1000)
        ticks.append(str((float(targets[scf][2])/1000+offset+float(targets[scf][1])/1000+offset)/2))
        print ticks[-1]
#print ticks
#print labels

#print data into the panels

print_arrows(filehandle=OUT, xmin=global_minimum, xmax=global_maximum, 
             targets_by_length=targets_by_length, targets=targets, support=2,
             windows=windows['Fst'], support_colors=['white', '#CD5C5C', 'black', '#B22222'])

for f in fst_files:
    print_Fst(filepath=f, filehandle=OUT, xmin=global_minimum, xmax=global_maximum, 
        targets_by_length=targets_by_length, targets=targets, min_support=min_support_for_print, 
        windows=windows['Eye'], support_colors=["0","#d4fbddff", "#a6f7baff", "#16e74dff"])

print_ARR(filepath=Eye_files[0], filehandle=OUT, xmin=global_minimum, xmax=global_maximum, 
    targets_by_length=targets_by_length, targets=targets, min_support=min_support_for_print, 
    windows=windows['Eye'], support_colors=["0","#d4fbddff", "#a6f7baff", "#16e74dff"], 
    xticks=ticks, xlabels=labels, global_avg=global_average, sd_max=sd_max)

OUT.write("\ndev.off()\n")
OUT.write("\n#reset to default\npar(default_par)")
OUT.close()


start: 71177
end: 18454481
scaffold_12
0.0
start: 71.177
end: 8693.769
4382.473
scaffold_39
8622.592
start: 80.328
end: 4871.528
11098.52
scaffold_148
13413.792
start: 43.527
end: 1696.972
14284.0415
scaffold_159
15067.237
start: 23.794
end: 1637.194
15897.731
scaffold_197
16680.637
start: 29.497
end: 952.411
17171.591
scaffold_215
17603.551
start: 135.232
end: 850.93
18096.632

### print arrows ###

preparing plot
print arrows

### Di_1-Di_2.tsv ###

preparing plot
printing background color and supported windows if applicable
plotting data

### Di_1-Di_4.tsv ###

preparing plot
printing background color and supported windows if applicable
plotting data

### Di_1-Di_5.tsv ###

preparing plot
printing background color and supported windows if applicable
plotting data

### Di_2-Di_4.tsv ###

preparing plot
printing background color and supported windows if applicable
plotting data

### Di_2-Di_5.tsv ###

preparing plot
printing background color and supported windows if applicable
plottin

In [144]:
!Rscript sliding_window_Eye_stringent-arrows.R

[1] "scaffold_12"
[1] "scaffold_39"
[1] "scaffold_148"
[1] "scaffold_159"
[1] "scaffold_197"
[1] "scaffold_215"
pdf 
  2 
Warning messages:
1: In par(default_par) : graphical parameter "cin" cannot be set
2: In par(default_par) : graphical parameter "cra" cannot be set
3: In par(default_par) : graphical parameter "csi" cannot be set
4: In par(default_par) : graphical parameter "cxy" cannot be set
5: In par(default_par) : graphical parameter "din" cannot be set
6: In par(default_par) : graphical parameter "page" cannot be set


Identify the SNPs and regions containing ARR >= 0.95 SNPs

In [267]:
top_95 = list(set(top_95))
print len(top_95)

33


In [271]:
#these are the scaffolds that will be processed
print "Candidates scaffolds: %s" %sorted(top_95)

targets = {}

#records all SNP coordinates for the scaffolds of interest
for p in pairs:
    print "processint %s" %p
    FH = open("../pairwise_Fst/"+p+".tsv", 'r')
    for l in FH:
#        print l.strip()
        if l.split("\t")[4] in top_95:
            if not l.split("\t")[4] in targets:
                targets[l.split("\t")[4]] = []
            targets[l.split("\t")[4]].append(int(l.split("\t")[5]))
            
    FH.close()

for k in targets.keys():
        targets[k] = sorted(list(set(targets[k])))                 

#print targets    
print "DONE"
                    
targets_by_length = {}
print targets.keys()
for t in sorted(targets):
    if not len(set(targets[t])) > 1:
        print "excluding %s" %t
        continue
#    print t,len(targets[t])
#    print sorted(targets[t])[0],sorted(targets[t])[-1]
    length=sorted(targets[t])[-1] - sorted(targets[t])[0]
    targets[t]=[str(length), str(sorted(targets[t])[0]), str(sorted(targets[t])[-1])]
    if not length in targets_by_length:
        targets_by_length[length] = []
    targets_by_length[length].append(t)
#    print targets[t]

longest=int(sorted(targets_by_length)[-1])
count=0


##
cummulative = 0
for t in reversed(sorted(targets_by_length.keys())):
#    print t
    for scf in targets_by_length[t]:
#        print scf
        
        targets[scf].append(str(cummulative))
        factor="%.2f" %(float(targets[scf][0])/longest)
        targets[scf].append(factor)
        print "%s\t%s" %(scf, "\t".join(targets[scf]))#, float(targets[scf][0])/longest)
        count+=1
        cummulative+=int(targets[scf][0])

print "Total number of scaffolds to print: %s" %count

Candidates scaffolds: ['scaffold_110', 'scaffold_12', 'scaffold_13', 'scaffold_145', 'scaffold_148', 'scaffold_149', 'scaffold_159', 'scaffold_16', 'scaffold_162', 'scaffold_19', 'scaffold_193', 'scaffold_197', 'scaffold_2', 'scaffold_21', 'scaffold_215', 'scaffold_23', 'scaffold_230', 'scaffold_236', 'scaffold_254', 'scaffold_258', 'scaffold_267', 'scaffold_269', 'scaffold_292', 'scaffold_30', 'scaffold_36', 'scaffold_361', 'scaffold_38', 'scaffold_39', 'scaffold_404', 'scaffold_44', 'scaffold_62', 'scaffold_74', 'scaffold_91']
processint Di_1-Di_2
processint Di_1-Di_4
processint Di_1-Di_5
processint Di_2-Di_4
processint Di_2-Di_5
processint Di_4-Di_5
DONE
['scaffold_269', 'scaffold_361', 'scaffold_267', 'scaffold_162', 'scaffold_145', 'scaffold_148', 'scaffold_149', 'scaffold_74', 'scaffold_16', 'scaffold_36', 'scaffold_13', 'scaffold_12', 'scaffold_39', 'scaffold_38', 'scaffold_19', 'scaffold_404', 'scaffold_110', 'scaffold_254', 'scaffold_159', 'scaffold_91', 'scaffold_258', 'scaff

In [272]:
regions = {}
windows = {}
dist=50000 #window size in bp (up and downstream of SNP)

In [273]:
to_process = ['Diplo_1M_smoothed_V_EY_D_p_TL', 'Diplo_1M_smoothed_H_Ey_D_p_TL']

#print candidates

regions['Eye'] = {}

#add placeholders for all candidate scaffolds
for c in top_95:
    regions['Eye'][c] = {}

#loop over the pairwise comparisons and record Eye candidate SNPs and their level of support
for p in to_process:
    out=[]
    ids=[]
    print "processint %s" %p
    FH = open(p+"-incl_support.txt", 'r')
    FH.readline()
    for l in FH:
        if l.split("\t")[0] in top_95: #if the scaffold is one of the candidates
            #extract the number of approaches supporting a given SNP as candidate locus
#            print "%s is a candidate tag" %l.split("\t")[1]
            support = l.strip().split("\t")[-1]
            if support > 0:
#                print "\n%s\t%s\t%s\t%s" %(l.split("\t")[1], l.split("\t")[4], l.split("\t")[5], support)

                for s in range(1,int(support)+1):
                    if not s in regions['Eye'][l.split("\t")[0]]:
                        regions['Eye'][l.split("\t")[0]][int(s)] = []
#                print regions
                for s in range(1,int(support)+1):
#                    print s
                    regions['Eye'][l.split("\t")[0]][s].append(int(l.split("\t")[1]))
#                print regions

        
    FH.close()   
    print "Done\n"

#identify candidate windows
windows['Eye'] = {}
for r in sorted(regions['Eye']):#regions):
    print r
    if not r in windows['Eye']:
        windows['Eye'][r]={}
    for s in regions['Eye'][r]:
        if not s in windows['Eye'][r]:
            windows['Eye'][r][s]=[]
#        print "\tsupport: %s" %s
        regions['Eye'][r][s]=sorted(list(set(regions['Eye'][r][s])))

        number = len(regions['Eye'][r][s])
#        print "\t\t %i candidate loci - %s" %(number, str(regions[r][s]))
        lower = regions['Eye'][r][s][0]-dist
        if lower < int(targets[r][1]):
            lower = int(targets[r][1])
        upper = regions['Eye'][r][s][0]+dist
        if upper > int(targets[r][2]):
            upper = int(targets[r][2])
        for i in range(len(regions['Eye'][r][s])-1):
#            print "\t\t%i\t%i" %(i,regions[r][s][i])
            if regions['Eye'][r][s][i] + dist >= regions['Eye'][r][s][i+1]-dist:
#                print "\t\t\t%i is in range - making %i the upper boundery" %(regions[r][s][i+1], regions[r][s][i+1]+dist)
                upper = regions['Eye'][r][s][i+1] + dist
            else:
#                print "\t\t\t%i is NOT in range" %regions[r][s][i+1]
                windows['Eye'][r][s].append([lower, upper])
                lower = regions['Eye'][r][s][i+1]-dist
                if lower < int(targets[r][1]):
                    lower = int(targets[r][1])
                upper = regions['Eye'][r][s][i+1]+dist
                if upper > int(targets[r][2]):
                    upper = int(targets[r][2])
        windows['Eye'][r][s].append([lower, upper])
#        print "\t\t %i window(s): %s" %(len(windows[r][s]),windows[r][s])
                

                
for t in reversed(sorted(targets_by_length.keys())):
#    print t
    for scf in targets_by_length[t]:
        print "\n### %s ###\n" %scf
        for i in range(1,len(windows['Eye'][scf])+1):
            print "level of support: %i" %i
            print "\t%i candidate loci - %s" %(len(regions['Eye'][scf][i]), regions['Eye'][scf][i])
            print "\t%i window(s) - %s" %(len(windows['Eye'][scf][i]), windows['Eye'][scf][i])

processint Diplo_1M_smoothed_V_EY_D_p_TL
Done

processint Diplo_1M_smoothed_H_Ey_D_p_TL
Done

scaffold_110
scaffold_12
scaffold_13
scaffold_145
scaffold_148
scaffold_149
scaffold_159
scaffold_16
scaffold_162
scaffold_19
scaffold_193
scaffold_197
scaffold_2
scaffold_21
scaffold_215
scaffold_23
scaffold_230
scaffold_236
scaffold_254
scaffold_258
scaffold_267
scaffold_269
scaffold_292
scaffold_30
scaffold_36
scaffold_361
scaffold_38
scaffold_39
scaffold_404
scaffold_44
scaffold_62
scaffold_74
scaffold_91

### scaffold_2 ###

level of support: 1
	16 candidate loci - [1272988, 2070518, 2105119, 2125974, 3891575, 4083295, 7529143, 9161690, 9243906, 10036157, 13641798, 14572160, 14658390, 14658478, 14709510, 14840534]
	10 window(s) - [[1222988, 1322988], [2020518, 2175974], [3841575, 3941575], [4033295, 4133295], [7479143, 7579143], [9111690, 9293906], [9986157, 10086157], [13591798, 13691798], [14522160, 14759510], [14790534, 14890534]]
level of support: 2
	1 candidate loci - [2070518]
	1 wi

In [289]:
SNP_count = 0
for scf in sorted(regions['Eye']):
    if int(2) in regions['Eye'][scf]:
        print scf,len(regions['Eye'][scf][2]),regions['Eye'][scf][2]
        SNP_count+=len(regions['Eye'][scf][2])

print "number of loci: %s" %SNP_count

window_count = 0
for scf in sorted(windows['Eye']):
    if int(2) in windows['Eye'][scf]:
        print scf,len(windows['Eye'][scf][2]),windows['Eye'][scf][2]
        window_count+=len(windows['Eye'][scf][2])

print "number of windows: %s" %window_count

scaffold_110 2 [623630, 2268317]
scaffold_12 2 [3799847, 7678845]
scaffold_13 2 [4995294, 5319891]
scaffold_145 1 [1007020]
scaffold_148 1 [543741]
scaffold_149 1 [1281289]
scaffold_159 1 [594470]
scaffold_16 1 [4481813]
scaffold_162 1 [162253]
scaffold_19 1 [3537015]
scaffold_193 3 [504998, 537410, 579879]
scaffold_197 3 [125383, 143069, 150472]
scaffold_2 1 [2070518]
scaffold_21 1 [3250318]
scaffold_215 1 [576905]
scaffold_23 1 [6086778]
scaffold_230 1 [735168]
scaffold_236 1 [96337]
scaffold_254 1 [347101]
scaffold_258 2 [454546, 566161]
scaffold_267 1 [76845]
scaffold_269 1 [510533]
scaffold_292 1 [437500]
scaffold_30 1 [1273610]
scaffold_36 1 [5276113]
scaffold_361 1 [62963]
scaffold_38 1 [4252011]
scaffold_39 2 [1277431, 1771937]
scaffold_404 1 [79400]
scaffold_44 2 [2691477, 2724350]
scaffold_62 1 [2613588]
scaffold_91 1 [2606509]
number of loci: 42
scaffold_110 2 [[573630, 673630], [2218317, 2273117]]
scaffold_12 2 [[3749847, 3849847], [7628845, 7728845]]
scaffold_13 2 [[494529

In [145]:
cd ..

/media/chrishah/STORAGE/RAD/popgen/sliding_window_plots/DIPLOTAXODON_FOR_PAPER


#global skyline plot

In [36]:
!mkdir global

In [311]:
cd global/

/media/chrishah/STORAGE/RAD/popgen/sliding_window_plots/DIPLOTAXODON_FOR_PAPER/global


Create symbolic links to relevant Fst files.

In [38]:
#create symbolic links to Fst data files
!ln -s ../pairwise_Fst/Di_* .

Create symblic link to relevant Eye morpho file.

In [39]:
!ln -s /media/chrishah/STORAGE/RAD/popgen/Bayenv/Diplotaxodon/4_populations/M_zebra-BWA-8MM-stacks_m5_n5_r_0.8_p4_ONLY_ONE_SNP/ANALYSES_FOR_DIPLOTAXODON_PAPER_EXCLUDE_SINGLETONS/RANK_STATISTIC_2/Diplo_1M_V_EY_D_p_TL.txt .

In [312]:
!ls

Di_1-Di_2.tsv		  Rplots.pdf
Di_1-Di_4.tsv		  sliding_window_global_arrows-color.png
Di_1-Di_5.tsv		  sliding_window_global_arrows-color.svg
Di_2-Di_4.tsv		  sliding_window_global_arrows.R
Di_2-Di_5.tsv		  sliding_window_global_arrows.svg
Di_4-Di_5.tsv		  sliding_window_global.R
Diplo_1M_V_EY_D_p_TL.txt  sliding_window_global.svg


Create a list of all scaffolds.

In [313]:
IN = open('Diplo_1M_V_EY_D_p_TL.txt','r')

candidate_scfs = []

print IN.readline()
for l in IN:
    candidate_scfs.append(l.split("\t")[0])
    
candidate_scfs = sorted(list(set(candidate_scfs)))
    


chrom	bp	SNPID	avg_rank	med_rank	p20_rank	std_rank	var_rank	mad_rank	avg_rank_rel	med_rank_rel	p20_rank_rel	var_rank_rel	var_rank_weight	var_weighted_avg_rank	var_weighted_rel_avg_rank



The next cell goes through the pairwise Fst files from populations and records the Fst candidate support and some other metrics of the scaffolds of interest.

In [314]:
#these are the scaffolds that will be processed
print "Candidates scaffolds: %s" %sorted(candidate_scfs)

targets = {}

#records all SNP coordinates for the scaffolds of interest
for p in pairs:
    print "processint %s" %p
    FH = open("../pairwise_Fst/"+p+".tsv", 'r')
    for l in FH:
#        print l.strip()
        if l.split("\t")[4] in candidate_scfs:
            if not l.split("\t")[4] in targets:
                targets[l.split("\t")[4]] = []
            targets[l.split("\t")[4]].append(int(l.split("\t")[5]))
            
    FH.close()

for k in targets.keys():
        targets[k] = sorted(list(set(targets[k])))                 

#print targets    
print "DONE"
                    
targets_by_length = {}
print targets.keys()
for t in sorted(targets):
    if not len(set(targets[t])) > 10:
        print "excluding %s" %t
        continue
#    print t,len(targets[t])
#    print sorted(targets[t])[0],sorted(targets[t])[-1]
    length=sorted(targets[t])[-1] - sorted(targets[t])[0]
    targets[t]=[str(length), str(sorted(targets[t])[0]), str(sorted(targets[t])[-1])]
    if not length in targets_by_length:
        targets_by_length[length] = []
    targets_by_length[length].append(t)
#    print targets[t]

longest=int(sorted(targets_by_length)[-1])
count=0


##
cummulative = 0
for t in reversed(sorted(targets_by_length.keys())):
#    print t
    for scf in targets_by_length[t]:
#        print scf
        
        targets[scf].append(str(cummulative))
        factor="%.2f" %(float(targets[scf][0])/longest)
        targets[scf].append(factor)
        print "%s\t%s" %(scf, "\t".join(targets[scf]))#, float(targets[scf][0])/longest)
        count+=1
        cummulative+=int(targets[scf][0])

print "Total number of scaffolds to print: %s" %count

Candidates scaffolds: ['scaffold_0', 'scaffold_1', 'scaffold_10', 'scaffold_100', 'scaffold_1009', 'scaffold_101', 'scaffold_102', 'scaffold_1020', 'scaffold_1022', 'scaffold_103', 'scaffold_104', 'scaffold_105', 'scaffold_106', 'scaffold_1063', 'scaffold_107', 'scaffold_108', 'scaffold_109', 'scaffold_1098', 'scaffold_11', 'scaffold_110', 'scaffold_1105', 'scaffold_111', 'scaffold_112', 'scaffold_1124', 'scaffold_113', 'scaffold_114', 'scaffold_115', 'scaffold_116', 'scaffold_117', 'scaffold_118', 'scaffold_119', 'scaffold_12', 'scaffold_120', 'scaffold_121', 'scaffold_1218', 'scaffold_122', 'scaffold_1229', 'scaffold_123', 'scaffold_124', 'scaffold_1247', 'scaffold_125', 'scaffold_126', 'scaffold_1269', 'scaffold_127', 'scaffold_128', 'scaffold_1282', 'scaffold_129', 'scaffold_13', 'scaffold_130', 'scaffold_131', 'scaffold_132', 'scaffold_133', 'scaffold_1332', 'scaffold_134', 'scaffold_135', 'scaffold_136', 'scaffold_137', 'scaffold_138', 'scaffold_1389', 'scaffold_139', 'scaffold_1

In [315]:
#These are the scaffolds to be printed
for t in reversed(sorted(targets_by_length.keys())):
    print targets_by_length[t],targets[targets_by_length[t][0]]

['scaffold_0'] ['18675497', '80418', '18755915', '0', '1.00']
['scaffold_1'] ['14915769', '39370', '14955139', '18675497', '0.80']
['scaffold_2'] ['14762014', '150830', '14912844', '33591266', '0.79']
['scaffold_3'] ['13934425', '47026', '13981451', '48353280', '0.75']
['scaffold_4'] ['12176843', '4075', '12180918', '62287705', '0.65']
['scaffold_5'] ['11993504', '64980', '12058484', '74464548', '0.64']
['scaffold_6'] ['10674875', '4280', '10679155', '86458052', '0.57']
['scaffold_7'] ['10634491', '24480', '10658971', '97132927', '0.57']
['scaffold_8'] ['10485647', '3709', '10489356', '107767418', '0.56']
['scaffold_9'] ['9940624', '380853', '10321477', '118253065', '0.53']
['scaffold_10'] ['9504148', '126961', '9631109', '128193689', '0.51']
['scaffold_13'] ['8650600', '21412', '8672012', '137697837', '0.46']
['scaffold_11'] ['8630161', '61405', '8691566', '146348437', '0.46']
['scaffold_12'] ['8622592', '71177', '8693769', '154978598', '0.46']
['scaffold_14'] ['8435196', '100244', '8

In [234]:


def print_Fst(filepath, filehandle, xmin, xmax, targets_by_length, targets, 
              min_support=0, windows=[], support_colors=[], xticks=[], xlabels=[]):
    """write R commands to print Fst, etc"""
    
    OUT=filehandle
    p=filepath
    
    print "\n### %s ###\n" %p
    OUT.write("\n# read files\n")
    OUT.write("%s <- read.delim(file = '%s', header = T, sep = \"\\t\")\n" %(p.replace("-","_"), p))
    OUT.write("\npop = %s\n" %p.replace("-","_"))

    print "preparing plot"
    OUT.write("plot(%s, 0, axes=T, cex=0.5, ylab = \"\", xlab = \"\", ylim = c(0,1), xlim = c(%s/1000,%s/1000), yaxt = 'n', xaxt = 'n', col = 'white')\n" %(xmin, xmin, xmax ))
    OUT.write("axis(side = 2, at = c(0,0.5,1), labels = T, las=1, cex.axis=0.8)\n")
    if xticks and xlabels:
#    if files_to_process.index(p) == len(files_to_process)-1:
        OUT.write("axis(side = 1, at = c(%s), labels = c(%s), las=3, cex.axis=0.8)\n" %(",".join(xticks), ",".join(xlabels)))

    print "printing ackground color and supported windows if applicable"
    c=0
    for t in list(reversed(sorted(targets_by_length.keys()))): #[:3]:
        c+=1
        for scf in targets_by_length[t]:  
#            print scf

            OUT.write("scf <- '%s'\n" %scf)
#            OUT.write("scf\n")
            offset = float(targets[scf][3])/1000
#            print "offset: %s" %offset
            length=int(targets[scf][2])-int(targets[scf][1])
#            print "length: %s" %length
            
            if c % 2 == 0 and length >= 1000000:
                OUT.write("rect(%s/1000+%s,0,%s/1000+%s,1, col = 'grey85', border = 'NA')\n" %(targets[scf][1], str(offset), targets[scf][2], str(offset)))
            else:
                OUT.write("rect(%s/1000+%s,0,%s/1000+%s,1, col = 'white', border = 'NA')\n" %(targets[scf][1], str(offset), targets[scf][2], str(offset)))
        
            if windows and min_support:
                for i in range(min_support, len(windows[scf])+1):
#                    print "supported by %i" %i
            
                    for j in range(len(windows[scf][i])):
#                        print "window %i: %s" %(j+1, windows[scf][i][j])
                        OUT.write("rect(%s/1000+%s,0,%s/1000+%s,1, col = '%s', border = '%s', lwd = '0.5')\n" %(windows[scf][i][j][0], str(offset), windows[scf][i][j][1], str(offset), support_colors[i], support_colors[i]))

    print "plotting data"
    for t in list(reversed(sorted(targets_by_length.keys()))): #[:3]:
        for scf in targets_by_length[t]:  
#            print scf
            OUT.write("scf <- '%s'\n" %scf)
#            OUT.write("scf\n")
            offset = float(targets[scf][3])/1000
#            print "offset: %s" %offset

            OUT.write("sub <- subset(pop, pop$Chr == scf)\n")
            OUT.write("#lines(sub$BP/1000+%s, sub$Smoothed.AMOVA.Fst, type='b', lty=1, pch=20, cex = 0.8, lwd = 0.6)\n" %str(offset))
            OUT.write("points(sub$BP/1000+%s, sub$AMOVA.Fst, cex=0.8, pch=20, ylab = \"\", xlab = \"\", yaxt = 'n', xaxt = 'n')\n" %(str(offset)) )

    OUT.write("abline(h=mean(pop$AMOVA.Fst), lty=5, lwd=1)\n")
    OUT.write("abline(h=0.5, lty=3, lwd=0.2)\n")
    OUT.write("abline(h=0, lty=1, lwd=0.2)\n")
    OUT.write("abline(h=1, lty=1, lwd=0.2)\n")
    

In [235]:
import numpy as np
import os

##################################################
#calculate maxiumum possible standard deviation
sd_max = max_sd(number=20, maxi=11786)

#calculate global weighted average for ARR from the full dataset
global_average = calculate_ARR_global_weighted_average(filepath='Diplo_1M_V_EY_D_p_TL.txt', sd_max=sd_max)

#################################################


#write R script

figure_prefix='sliding_window_global'
fst_files = [x+".tsv" for x in pairs]
#Eye_files = ['Diplo_1M_V_EY_D_p_TL.txt'] #prefix of the files to be processed

number_of_panels_vertical = 6
number_of_panels_horizontal = 1
wd = os.path.abspath(".") #working directory
#category = 'Eye' #support category to highlight
#support_colors=["0","#d4fbddff", "#a6f7baff", "#16e74dff"]
#min_support_for_print = 2


OUT=open(figure_prefix+'.R','w')
OUT.write("")

#set up dimensions of plot
matr=",".join([ str(x) for x in range(1,(number_of_panels_horizontal * number_of_panels_vertical)+1) ])
widths="1" #,".join(width)
heigths=",".join( [ str(1) for x in range(number_of_panels_horizontal * number_of_panels_vertical)])
#print matr
#print widths
#print heigths

OUT.write("#record the default parameters for plotting\ndefault_par <- par()\n\n")

OUT.write("\n# set working directory\nsetwd(dir = '%s')\n" %wd)
#OUT.write("\n# read files\n")
#for p in panel_prefix:
#    OUT.write("%s <- read.delim(file = '%s', header = T, sep = \"\\t\")\n" %(p.replace("-","_"), p))
    
OUT.write("\nsvg(filename = '%s.svg')\n" %figure_prefix)
OUT.write("mat <- matrix(c(%s), %i)\n" %(matr, number_of_panels_vertical))
OUT.write("layout(mat, widths=c(%s), heights=c(%s))\n" %(widths, heigths))
OUT.write("par(mar=c(0.2, 0, 0, 0), oma = c(1,3,1,1))\n")


#determine global minimum and maximum on the x axis of the plot
global_minimum = int(int(targets[targets_by_length[list(reversed(sorted(targets_by_length.keys())))[0]][0]][1]) + int(targets[targets_by_length[list(reversed(sorted(targets_by_length.keys())))[0]][0]][-2]))
print "start: %s" %global_minimum
global_maximum = int(int(targets[targets_by_length[list(reversed(sorted(targets_by_length.keys())))[-1]][0]][2]) + int(targets[targets_by_length[list(reversed(sorted(targets_by_length.keys())))[-1]][0]][-2]))
print "end: %s" %global_maximum


#determine positions for xticks and xlabels
ticks=[]
labels=[]
for t in list(reversed(sorted(targets_by_length.keys()))): #[:3]:
    for scf in targets_by_length[t]:
#        print scf
        offset = float(targets[scf][3])/1000
#        print offset
        labels.append(str(scf.split("_")[1]))
#        print "start: %s" %str(float(targets[scf][1])/1000)
#        print "end: %s" %str(float(targets[scf][2])/1000)
        ticks.append(str((float(targets[scf][2])/1000+offset+float(targets[scf][1])/1000+offset)/2))
#        print ticks[-1]
#print ticks
#print labels

#print data into the panels

for f in fst_files[:]:
    print_Fst(filepath=f, filehandle=OUT, xmin=global_minimum, xmax=global_maximum, 
        targets_by_length=targets_by_length, targets=targets)

#print_Fst(filepath=f, filehandle=OUT, xmin=global_minimum, xmax=global_maximum, 
#        targets_by_length=targets_by_length, targets=targets,
#        xticks=ticks, xlabels=labels)

OUT.write("\ndev.off()\n")
OUT.write("\n#reset to default\npar(default_par)")
OUT.close()


start: 80418
end: 735652334

### Di_1-Di_2.tsv ###

preparing plot
printing ackground color and supported windows if applicable
plotting data

### Di_1-Di_4.tsv ###

preparing plot
printing ackground color and supported windows if applicable
plotting data

### Di_1-Di_5.tsv ###

preparing plot
printing ackground color and supported windows if applicable
plotting data

### Di_2-Di_4.tsv ###

preparing plot
printing ackground color and supported windows if applicable
plotting data

### Di_2-Di_5.tsv ###

preparing plot
printing ackground color and supported windows if applicable
plotting data

### Di_4-Di_5.tsv ###

preparing plot
printing ackground color and supported windows if applicable
plotting data


In [236]:
!Rscript sliding_window_global.R

pdf 
  2 
Warning messages:
1: In par(default_par) : graphical parameter "cin" cannot be set
2: In par(default_par) : graphical parameter "cra" cannot be set
3: In par(default_par) : graphical parameter "csi" cannot be set
4: In par(default_par) : graphical parameter "cxy" cannot be set
5: In par(default_par) : graphical parameter "din" cannot be set
6: In par(default_par) : graphical parameter "page" cannot be set


In [316]:


#print candidates

regions['Fst'] = {}

#add placeholders for all candidate scaffolds
for c in candidate_scfs:
    regions['Fst'][c] = {}

#loop over the pairwise comparisons and record Fst candidate SNPs and their level of support
for p in pairs:
    out=[]
    ids=[]
    print "processint %s" %p
    FH = open(p+".tsv", 'r')
    FH.readline()
    for l in FH:
        if l.split("\t")[4] in candidate_scfs: #if the scaffold is one of the candidates
            #extract the number of approaches supporting a given SNP as candidate locus
#            print "%s is a candidate tag" %l.split("\t")[1]
            support = l.strip().split("\t")[-1]
            if support > 0:
#                print "\n%s\t%s\t%s\t%s" %(l.split("\t")[1], l.split("\t")[4], l.split("\t")[5], support)

                for s in range(1,int(support)+1):
                    if not s in regions['Fst'][l.split("\t")[4]]:
                        regions['Fst'][l.split("\t")[4]][int(s)] = []
#                print regions
                for s in range(1,int(support)+1):
#                    print s
                    regions['Fst'][l.split("\t")[4]][s].append(int(l.split("\t")[5]))
#                print regions

        
    FH.close()   
    print "Done\n"

#identify candidate windows
windows['Fst'] = {}
for t in reversed(sorted(targets_by_length.keys())):
    r = targets_by_length[t][0]

#for r in sorted(targets):#regions):
#    print r
    if not r in windows['Fst']:
        windows['Fst'][r]={}
    for s in regions['Fst'][r].keys():
        if not s in windows['Fst'][r]:
            windows['Fst'][r][s]=[]
#        print "\tsupport: %s" %s
        regions['Fst'][r][s]=sorted(list(set(regions['Fst'][r][s])))

        number = len(regions['Fst'][r][s])
#        print "\t\t %i candidate loci - %s" %(number, str(regions['Fst'][r][s]))
        lower = regions['Fst'][r][s][0]-dist
#        print targets[r]
        if lower < int(targets[r][1]):
            lower = int(targets[r][1])
        upper = regions['Fst'][r][s][0]+dist
        if upper > int(targets[r][2]):
            upper = int(targets[r][2])
        for i in range(len(regions['Fst'][r][s])-1):
#            print "\t\t%i\t%i" %(i,regions[r][s][i])
            if regions['Fst'][r][s][i] + dist >= regions['Fst'][r][s][i+1]-dist:
#                print "\t\t\t%i is in range - making %i the upper boundery" %(regions[r][s][i+1], regions[r][s][i+1]+dist)
                upper = regions['Fst'][r][s][i+1] + dist
            else:
#                print "\t\t\t%i is NOT in range" %regions[r][s][i+1]
                windows['Fst'][r][s].append([lower, upper])
                lower = regions['Fst'][r][s][i+1]-dist
                if lower < int(targets[r][1]):
                    lower = int(targets[r][1])
                upper = regions['Fst'][r][s][i+1]+dist
                if upper > int(targets[r][2]):
                    upper = int(targets[r][2])
        windows['Fst'][r][s].append([lower, upper])
#        print "\t\t %i window(s): %s" %(len(windows[r][s]),windows[r][s])
                

                
for t in reversed(sorted(targets_by_length.keys())):
#    print t
    for scf in targets_by_length[t]:
        print "\n### %s ###\n" %scf
        for i in range(1,len(windows['Fst'][scf])+1):
            print "supported by %i approach(es):" %i
            print "\t%i candidate loci - %s" %(len(regions['Fst'][scf][i]), regions['Fst'][scf][i])
            print "\t%i window(s) - %s" %(len(windows['Fst'][scf][i]), windows['Fst'][scf][i])

processint Di_1-Di_2
Done

processint Di_1-Di_4
Done

processint Di_1-Di_5
Done

processint Di_2-Di_4
Done

processint Di_2-Di_5
Done

processint Di_4-Di_5
Done


### scaffold_0 ###

supported by 1 approach(es):
	2 candidate loci - [8979100, 12990331]
	2 window(s) - [[8929100, 9029100], [12940331, 13040331]]

### scaffold_1 ###

supported by 1 approach(es):
	2 candidate loci - [330682, 6150343]
	2 window(s) - [[280682, 380682], [6100343, 6200343]]

### scaffold_2 ###

supported by 1 approach(es):
	2 candidate loci - [4083294, 13641797]
	2 window(s) - [[4033294, 4133294], [13591797, 13691797]]

### scaffold_3 ###

supported by 1 approach(es):
	1 candidate loci - [6501561]
	1 window(s) - [[6451561, 6551561]]

### scaffold_4 ###

supported by 1 approach(es):
	1 candidate loci - [1868067]
	1 window(s) - [[1818067, 1918067]]

### scaffold_5 ###

supported by 1 approach(es):
	7 candidate loci - [4388811, 4390464, 4392228, 4392261, 4392300, 4392374, 4429337]
	1 window(s) - [[4338811, 4479337]

In [238]:
def print_arrows(filehandle, xmin, xmax, targets_by_length, targets, support=0, windows=[], support_colors=[]):
    """write R commands to print arrows"""
    
    min_support = support
    
    OUT=filehandle
    print "\n### print arrows ###\n"
    OUT.write("library(shape)\n")
    print "preparing plot"
    OUT.write("plot(%s, 0, axes=F, cex=0.5, ylab = \"\", xlab = \"\", ylim = c(0,1), xlim = c(%s/1000,%s/1000), yaxt = 'n', xaxt = 'n', col = 'white')\n" %(xmin, xmin, xmax ))

    print "print arrows"
    for i in range(min_support, 4):
#        print "supported by %i" %i
        support = i
        for scf in windows.keys(): #these are all scaffolds that contain a candidates SNP
            if len(windows[scf]) > 0:
#                print scf
                offset = float(targets[scf][3])/1000
#                print "offset: %s" %offset
                if support in windows[scf].keys():
                    for j in range(len(windows[scf][support])):
#                        print "window %i: %s" %(j+1, windows[scf][i][j])
                        center = np.average([windows[scf][support][j][0], windows[scf][support][j][1]])
                        OUT.write("#rect(%s/1000+%s,0,%s/1000+%s,1, col = '%s', border = '%s', lwd = '0.5')\n" %(windows[scf][support][j][0], str(offset), windows[scf][support][j][1], str(offset), support_colors[i], support_colors[i]))
                        OUT.write("center <- %s/1000\n" %center)
                    
                        OUT.write("Arrows(x0=center+%s, y0=1, x1=center+%s, y1=0, col='%s', arr.adj = 1, arr.width=0.5, arr.type='triangle')\n" %(offset, offset, support_colors[support]))

In [243]:
import numpy as np
import os


#write R script

figure_prefix='sliding_window_global_arrows'
fst_files = [x+".tsv" for x in pairs]
#Eye_files = ['Diplo_1M_V_EY_D_p_TL.txt'] #prefix of the files to be processed

number_of_panels_vertical = 7
number_of_panels_horizontal = 1
wd = os.path.abspath(".") #working directory
#category = 'Eye' #support category to highlight
#support_colors=["0","#d4fbddff", "#a6f7baff", "#16e74dff"]
#min_support_for_print = 2


OUT=open(figure_prefix+'.R','w')
OUT.write("")

#set up dimensions of plot
matr=",".join([ str(x) for x in range(1,(number_of_panels_horizontal * number_of_panels_vertical)+1) ])
widths="1" #,".join(width)
heigths = "0.2," #the height of the first panel
heigths+=",".join( [ str(1) for x in range(number_of_panels_horizontal * number_of_panels_vertical - 1)])

#print matr
#print widths
#print heigths

OUT.write("#record the default parameters for plotting\ndefault_par <- par()\n\n")

OUT.write("\n# set working directory\nsetwd(dir = '%s')\n" %wd)
#OUT.write("\n# read files\n")
#for p in panel_prefix:
#    OUT.write("%s <- read.delim(file = '%s', header = T, sep = \"\\t\")\n" %(p.replace("-","_"), p))
    
OUT.write("\nsvg(filename = '%s.svg')\n" %figure_prefix)
OUT.write("mat <- matrix(c(%s), %i)\n" %(matr, number_of_panels_vertical))
OUT.write("layout(mat, widths=c(%s), heights=c(%s))\n" %(widths, heigths))
OUT.write("par(mar=c(0.2, 0, 0, 0), oma = c(1,3,1,1))\n")


#determine global minimum and maximum on the x axis of the plot
global_minimum = int(int(targets[targets_by_length[list(reversed(sorted(targets_by_length.keys())))[0]][0]][1]) + int(targets[targets_by_length[list(reversed(sorted(targets_by_length.keys())))[0]][0]][-2]))
print "start: %s" %global_minimum
global_maximum = int(int(targets[targets_by_length[list(reversed(sorted(targets_by_length.keys())))[-1]][0]][2]) + int(targets[targets_by_length[list(reversed(sorted(targets_by_length.keys())))[-1]][0]][-2]))
print "end: %s" %global_maximum


#determine positions for xticks and xlabels
ticks=[]
labels=[]
for t in list(reversed(sorted(targets_by_length.keys()))): #[:3]:
    for scf in targets_by_length[t]:
#        print scf
        offset = float(targets[scf][3])/1000
#        print offset
        labels.append(str(scf.split("_")[1]))
#        print "start: %s" %str(float(targets[scf][1])/1000)
#        print "end: %s" %str(float(targets[scf][2])/1000)
        ticks.append(str((float(targets[scf][2])/1000+offset+float(targets[scf][1])/1000+offset)/2))
#        print ticks[-1]
#print ticks
#print labels

#print data into the panels

print_arrows(filehandle=OUT, xmin=global_minimum, xmax=global_maximum, 
             targets_by_length=targets_by_length, targets=targets, support=3,
             windows=windows['Fst'], support_colors=["0","#d4fbddff", "#a6f7baff", "#16e74dff"])

for f in fst_files[:]:
    print_Fst(filepath=f, filehandle=OUT, xmin=global_minimum, xmax=global_maximum, 
        targets_by_length=targets_by_length, targets=targets)

#print_Fst(filepath=f, filehandle=OUT, xmin=global_minimum, xmax=global_maximum, 
#        targets_by_length=targets_by_length, targets=targets,
#        xticks=ticks, xlabels=labels)

OUT.write("\ndev.off()\n")
OUT.write("\n#reset to default\npar(default_par)")
OUT.close()


start: 80418
end: 735652334

### print arrows ###

preparing plot
print arrows

### Di_1-Di_2.tsv ###

preparing plot
printing ackground color and supported windows if applicable
plotting data

### Di_1-Di_4.tsv ###

preparing plot
printing ackground color and supported windows if applicable
plotting data

### Di_1-Di_5.tsv ###

preparing plot
printing ackground color and supported windows if applicable
plotting data

### Di_2-Di_4.tsv ###

preparing plot
printing ackground color and supported windows if applicable
plotting data

### Di_2-Di_5.tsv ###

preparing plot
printing ackground color and supported windows if applicable
plotting data

### Di_4-Di_5.tsv ###

preparing plot
printing ackground color and supported windows if applicable
plotting data


In [244]:
!Rscript sliding_window_global_arrows.R

pdf 
  2 
Warning messages:
1: In par(default_par) : graphical parameter "cin" cannot be set
2: In par(default_par) : graphical parameter "cra" cannot be set
3: In par(default_par) : graphical parameter "csi" cannot be set
4: In par(default_par) : graphical parameter "cxy" cannot be set
5: In par(default_par) : graphical parameter "din" cannot be set
6: In par(default_par) : graphical parameter "page" cannot be set


##Extract basic stats for paper (number of genomic regions, etc.)

In [414]:


#print candidates

regions['Fst'] = {}

#add placeholders for all scaffolds
#for c in candidate_scfs:
#    regions['Fst'][c] = {}

    
id_seen = []
scfs=[]
#loop over the pairwise comparisons and record Fst candidate SNPs and their level of support
for p in pairs:
    out=[]
    ids=[]
    
    print "processint %s" %p
    FH = open(p+".tsv", 'r')
    FH.readline()
    for l in FH:
#        if l.split("\t")[4] in candidate_scfs: #if the scaffold is one of the candidates
            #extract the number of approaches supporting a given SNP as candidate locus
#            print "%s is a candidate tag" %l.split("\t")[1]
            support = int(l.strip().split("\t")[-1])
            if support > 0:
                if l.split("\t")[1] in id_seen:
                    continue
                scfs.append(l.split("\t")[4])
                id_seen.append(l.split("\t")[1])
                if not l.split("\t")[4] in regions['Fst']:
                    regions['Fst'][l.split("\t")[4]] = {}
                    scfs.append(l.split("\t")[4])
#                print "\n%s\t%s\t%s\t%s" %(l.split("\t")[1], l.split("\t")[4], l.split("\t")[5], support)

                for s in range(1,int(support)+1):
                    if not s in regions['Fst'][l.split("\t")[4]]:
                        regions['Fst'][l.split("\t")[4]][int(s)] = []
#                print regions
                for s in range(1,int(support)+1):
#                    print s
                    regions['Fst'][l.split("\t")[4]][s].append(int(l.split("\t")[5]))
#                print regions
#    print len(id_seen)
        
    FH.close()   
    print "Done\n"

    
print len(list(set(scfs)))
print len(regions['Fst'].keys())    
#print len(id_seen)
#print len(regions['Fst'])
    
#for scf in regions['Fst'].keys():
#    if not regions['Fst'][scf]:
#        del regions['Fst'][scf]
#    else:
#        for support in regions['Fst'][scf].keys():
#            regions['Fst'][scf][support] = sorted(list(set(regions['Fst'][scf][support])))

#print len(regions['Fst'])


#identify candidate windows
windows['Fst'] = {}
for t in reversed(sorted(regions['Fst'].keys())):
    r = t

#for r in sorted(targets):#regions):
    print r
    print regions['Fst'][r]
    if not r in windows['Fst']:
        windows['Fst'][r]={}
    for s in regions['Fst'][r].keys():
        if not s in windows['Fst'][r]:
            windows['Fst'][r][s]=[]
#        print targets[r]
#        print "\tsupport: %s" %s
        regions['Fst'][r][s]=sorted(list(set(regions['Fst'][r][s])))

        number = len(regions['Fst'][r][s])
#        print "\t\t %i candidate loci - %s" %(number, str(regions['Fst'][r][s]))

#        print targets[r]
#        if not len(targets[r]) > 2:
#            windows['Fst'][r][s].append(['dummy'])
#            print "DUMMY"
#            continue
            
        lower = regions['Fst'][r][s][0]-dist

        if lower < int(0):
            lower = int(0)
        upper = regions['Fst'][r][s][0]+dist
        for i in range(len(regions['Fst'][r][s])-1):
#            print "\t\t%i\t%i" %(i,regions[r][s][i])
            if regions['Fst'][r][s][i] + dist >= regions['Fst'][r][s][i+1]-dist:
#                print "\t\t\t%i is in range - making %i the upper boundery" %(regions[r][s][i+1], regions[r][s][i+1]+dist)
                upper = regions['Fst'][r][s][i+1] + dist
            else:
#                print "\t\t\t%i is NOT in range" %regions[r][s][i+1]
                windows['Fst'][r][s].append([lower, upper])
                lower = regions['Fst'][r][s][i+1]-dist
                if lower < int(targets[r][1]):
                    lower = int(targets[r][1])
                upper = regions['Fst'][r][s][i+1]+dist
                if upper > int(targets[r][2]):
                    upper = int(targets[r][2])
        windows['Fst'][r][s].append([lower, upper])
#        print "\t\t %i window(s): %s" %(len(windows['Fst'][r][s]),windows['Fst'][r][s])
                

            
for scf in sorted(regions['Fst'].keys()):
    print "\n### %s ###\n" %scf
    for i in range(1,len(windows['Fst'][scf])+1):
        print "supported by %i approach(es):" %i
        print "\t%i candidate loci - %s" %(len(regions['Fst'][scf][i]), regions['Fst'][scf][i])
        print "\t%i window(s) - %s" %(len(windows['Fst'][scf][i]), windows['Fst'][scf][i])

processint Di_1-Di_2
Done

processint Di_1-Di_4
Done

processint Di_1-Di_5
Done

processint Di_2-Di_4
Done

processint Di_2-Di_5
Done

processint Di_4-Di_5
Done

103
103
scaffold_96
{1: [317007, 363059, 2001386, 2569737]}
scaffold_92
{1: [600451, 882844, 919383]}
scaffold_900
{1: [9378]}
scaffold_9
{1: [1045335]}
scaffold_89
{1: [2637630]}
scaffold_88
{1: [1647236]}
scaffold_87
{1: [2371380]}
scaffold_85
{1: [771961, 719224]}
scaffold_83
{1: [1876874]}
scaffold_82
{1: [1571202]}
scaffold_81
{1: [368634, 374832, 388571, 450718], 2: [374832, 450718], 3: [374832, 450718]}
scaffold_80
{1: [429589]}
scaffold_8
{1: [7183865, 7109113]}
scaffold_79
{1: [617123, 875277]}
scaffold_74
{1: [2770537]}
scaffold_72
{1: [2133009]}
scaffold_71
{1: [2203118, 2157614]}
scaffold_7
{1: [7739905, 7661153]}
scaffold_61
{1: [81475, 113297, 113184]}
scaffold_58
{1: [984758, 942908, 942816, 944393], 2: [942816]}
scaffold_56
{1: [3282442]}
scaffold_55
{1: [1871822, 3795438, 1867395, 1871914], 2: [1871822]}
scaff

In [416]:
print len(regions['Fst'])
print len(windows['Fst'])



103
103


In [418]:

for i in range(1,4):
    print "\n#########\n######## Support level: %s #########\n#################\n" %i
    SNP_count=0
    window_count=0
    scf_count=0
    
    for scf in sorted(regions['Fst'].keys()):
        if i in regions['Fst'][scf]:
#            print "\n### %s ###\n" %scf
#            print "%i loci - %s" %(len(regions['Fst'][scf][i]), regions['Fst'][scf][i])
            
#        for i in range(1,len(windows['Fst'][scf])+1):
#            print "supported by %i approach(es):" %i
#            print "\t%i candidate loci - %s" %(len(regions['Fst'][scf][i]), regions['Fst'][scf][i])
            SNP_count+=len(regions['Fst'][scf][i])
#            print "%i window(s) - %s" %(len(windows['Fst'][scf][i]), windows['Fst'][scf][i])
            window_count+=len(windows['Fst'][scf][i])
            scf_count+=1

    print "Number of SNPs: %s" %SNP_count
    print "Number of scaffolds: %s" %scf_count
    print "Number of windows: %s" %window_count


#########
######## Support level: 1 #########
#################

Number of SNPs: 243
Number of scaffolds: 103
Number of windows: 134

#########
######## Support level: 2 #########
#################

Number of SNPs: 41
Number of scaffolds: 26
Number of windows: 26

#########
######## Support level: 3 #########
#################

Number of SNPs: 13
Number of scaffolds: 8
Number of windows: 8


In [253]:
cd ..

/media/chrishah/STORAGE/RAD/popgen/sliding_window_plots/DIPLOTAXODON_FOR_PAPER
